### Imports

In [1]:
#import import_ipynb
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
#from baseline_models import get_words, get_nouns, get_noun_pairs, get_used_noun_frequency

### old imports from base line models
import pandas as pd
import numpy as np
import ast
import random
import re

from collections import Counter, defaultdict 

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt

import inflect
p = inflect.engine()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thrdl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Using TensorFlow backend.


### Code

In [54]:
def prepare_text_keep_posts(df):
    texts = df.text.to_list() 
    
    posts = []
    for sublist in texts:
        sentences = []
        sentence = []
        for word in sublist:
            word = word.lower()
            if "n't" in word:
                word = "not"
            pattern = re.compile(r'[\"\'`,;]')
            if pattern.findall(word):
                pass
            else:
                pattern = re.compile(r'[\.\?\!]')
                if pattern.findall(word):
                    if len(sentence) > 1:
                        sentences.append(" ".join(sentence))
                    sentence = []
                else:
                    flag = True
                    for char in word:
                        if ord(char) > 127:
                            flag = False
                            break
                    if flag:
                        sentence.append(word)
        if len(sentence) > 0:
            sentences.append(" ".join(sentence))
        posts.append(sentences)

    return posts

def prepare_text(df):
    texts = df.text.to_list() 
    
    sentences = []
    for sublist in texts:
        sentence = []
        for word in sublist:
            word = word.lower()
            if "n't" in word:
                word = "not"
            pattern = re.compile(r'[\"\'`]')
            if pattern.findall(word):
                pass
            else:
                pattern = re.compile(r'[\.\?\!;,]')
                if pattern.findall(word):
                    if len(sentence) > 1:
                        sentences.append(" ".join(sentence))
                    sentence = []
                else:
                    flag = True
                    for char in word:
                        if ord(char) > 127:
                            flag = False
                            break
                    if flag:
                        sentence.append(word)
        if len(sentence) > 0:
            sentences.append(" ".join(sentence))

    return sentences

def tensor_label_array(embed,arrays_of_sentences_1, arrays_of_sentences_2):
    tensor_label = []
    for embedding, sentence in zip(embed, arrays_of_sentences_1):
        if sentence in arrays_of_sentences_2:
            tensor_label.append([embedding,1])
        else:
            tensor_label.append([embedding,0])
    #print(tensor_label[:5])
    return tensor_label

def get_words(df):
    sentences = df.text.to_list()    
    flat_list = []
    for sublist in sentences:
        for item in sublist:
            flat_list.append(item)           
    return flat_list

def get_noun_pairs(words):
    nouns = []
    noun_pairs = []
    porter = PorterStemmer()
    for i, (word, tag) in enumerate(words):
        if "n't" in word:
                word = "not"
        
        flag = True
        for char in word:
            if ord(char) > 127:
                flag = False
                break
        if len(word) >= 3 and tag.startswith('NN') and flag:
            if i > 0 and tag.startswith('NN') and (words[i-1][1].startswith('NN') or words[i-1][1].startswith('JJ') or words[i-1][1].startswith('PRP')):
                w1 = word
                w0 = ""
                for char in words[i-1][0]:
                    if ord(char) < 128:
                        w0 += char
                noun_pairs.append(w0+" "+w1)

            else:
                nouns.append(word)
                #nouns.append(porter.stem(word)) #convert words to their basic form
    return nouns, noun_pairs

def evaluate(tensor_label, df, label):
    accuracy = 0
    
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="default",
                           trainable=False)
    m = tf.keras.metrics.CosineSimilarity(axis=0)
    
    ctn = 0
    for row in df.iterrows():
        print(f"word: {ctn}")
        ctn +=1
        
        words_1 = get_words(val_1)
        tags_1 = nltk.pos_tag(words_1)
        nouns_1, pairs_1 = get_noun_pairs(tags_1)
        total_len = 0
        
        if len(pairs_1) > 0:
            mixture = np.array(pairs_1, dtype=bytes)
            total_len = len(pairs_1)
        else:
            mixture = np.array(nouns_1, dtype=bytes)
            total_len = len(nouns_1)
        
    
        embed_all = hub_layer(mixture)
        embed_all = tf.slice(embed_all, [0, 0], [total_len, 300])
        
        vote = 0
        for embed in embed_all:
            cos_max = -1
            label_vote = 0
            for tensors in tensor_label:
                m = tf.keras.metrics.CosineSimilarity(axis=0)
                m.update_state(embed, tensors[0])
                new_cos = m.result().numpy()
                if new_cos > cos_max:
                    cos_max = new_cos
                    label_vote = tensors[1]
            if label_vote == 0:
                vote -= 1
            else:
                vote += 1
        if (label == 0 and vote < 0) or (label == 1 and vote >= 0):
            accuracy += 1
    return accuracy

def evaluate_np(tensor_label, df, label):
    accuracy = 0
    posts = prepare_text_keep_posts(df)
    m = tf.keras.metrics.CosineSimilarity(axis=0)
    
    ctn = 0
    for post in posts:
        print(f"ctn: {ctn}")
        ctn += 1
        vote = 0
        
        embeds = get_all_embeddings(post,label)
        
        
        
        if len(embeds) < 1 and label == 0:
            accuracy += 1
        
        for embed in embeds:
            cos_max = -1
            label_vote = 0
            index = 0
            
            for j, tensors in enumerate(tensor_label):
                m.update_state(embed[0], tensors[0])
                new_cos = m.result().numpy()
                if new_cos > cos_max:
                    cos_max = new_cos
                    label_vote = tensors[1]
                    index = j
            if label_vote == 0:
                vote -= 1
            else:
                vote += 1
            if cos_max > 0.25:
                print(f"{cos_max}, {embed[2]} : {tensor_label[index][2]}")
                print("----------")
        if (label == 0 and vote < 0) or (label == 1 and vote >= 0):
            accuracy += 1
    
    return accuracy

def get_embedding_of_noun_pairs(sentence, label, model, hub_layer):
    words_to_tag = sentence.split()
    words = nltk.pos_tag(words_to_tag)
    noun_pair_positions = []    
    noun_pairs = []
    
    for i, (word, tag) in enumerate(words):
        if len(word) >= 3 and tag.startswith('NN'):
            if i > 0: #and tag.startswith('NN') and (words[i-1][1].startswith('NN') or words[i-1][1].startswith('JJ') or words[i-1][1].startswith('PRP')):
                w1 = word
                w0 = ""
                for char in words[i-1][0]:
                    if ord(char) < 128:
                        w0 += char
                noun_pairs.append(w0+" "+w1)
                noun_pair_positions.append([i-1, i])
                
    
    #print(sentence)
    
    mixture = np.array([sentence], dtype=bytes)
    #print(mixture)
    
    embed = hub_layer(mixture)
    
    noun_embeddings = []
    for i, pair in enumerate(noun_pair_positions):
        t1 = tf.slice(embed[0][pair[0]], [0], [50])
        t2 = tf.slice(embed[0][pair[1]], [0], [250])
        t_res = tf.concat([t1, t2],0) 
    
        noun_embeddings.append([t_res, label, noun_pairs[i]])
    return noun_embeddings
    
    
def get_all_embeddings(sentences, label):
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="elmo",
                           trainable=False)
    
    noun_embeddings = []
    for i, sentence in enumerate(sentences[:]):
        print(f"s_{i}")
        noun_embeddings.extend(get_embedding_of_noun_pairs(sentence, label, model, hub_layer))
    return noun_embeddings

In [ ]:
topics = { #"abortion": ["abortion_pro_choice.csv", "abortion_pro_life.csv"], #validation err: 0.5416666666666666
           "gay_marriage":["gay_marriage_for.csv", "gay_marriage_against.csv"],
           #"darwin_theory_of_evolution" :["darwin_theory_of_evolution_for.csv", "darwin_theory_of_evolution_against.csv"],
          #"marijuana_legalization" :["marijuana_legalization_against.csv", "marijuana_legalization_for.csv"], #validation err: 0.6777777777777778
         }


for key, pair in topics.items():

    df_1 = pd.read_csv(f"./dataset_processed/{pair[0]}", converters={2:ast.literal_eval})
    df_2 = pd.read_csv(f"./dataset_processed/{pair[1]}", converters={2:ast.literal_eval})
    
    tr_1, ts_1 = train_test_split(df_1, test_size=0.1, random_state=42)
    tr_2, ts_2 = train_test_split(df_2, test_size=0.1, random_state=42)
    
    tr_1, val_1 = train_test_split(tr_1, test_size=0.25, random_state=25)
    tr_2, val_2 = train_test_split(tr_2, test_size=0.25, random_state=25)
    
    arrays_of_sentences_1 = prepare_text(tr_1)
    arrays_of_sentences_2 = prepare_text(tr_2)
    
    embeddings_1  = get_all_embeddings(arrays_of_sentences_1, 0)
    embeddings_2  = get_all_embeddings(arrays_of_sentences_2, 1)
    
    #_1 extended with _2 = not needed
    embeddings_1.extend(embeddings_2)
    print(len(embeddings_1))
    
    acc_1 = evaluate_np(embeddings_1, val_1, 0)
    acc_2 = evaluate_np(embeddings_1, val_2, 1)
    
    print(f"validation err: {(acc_1+acc_2)/(len(val_1.index)+len(val_2.index))}")
    """
    
    arrays_of_sentences_1.extend(arrays_of_sentences_2)
    random.shuffle(arrays_of_sentences_1)
    
    
    #mixture = arrays_of_sentences_1
    mixture = np.array(arrays_of_sentences_1, dtype=bytes)
    
    print(len(mixture))

    
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="default",
                           trainable=False)
    
    embed_all = hub_layer(mixture[:100])
    embed_all = tf.slice(embed_all, [0, 0], [100, 300]) 
    
    for i in range(100, len(mixture)-200, 200):
        print(i)
        embed = hub_layer(mixture[i:i+200])
        embed = tf.slice(embed, [0, 0], [200, 300]) 
        embed_all = tf.concat([embed_all, embed], 0) 
    print("executed")
    tensor_label = tensor_label_array(embed,arrays_of_sentences_1, arrays_of_sentences_2)
    
    acc_1 = evaluate(tensor_label, val_1, 0)
    acc_2 = evaluate(tensor_label, val_2, 1)
    
    print(f"validation err: {(acc_1+acc_2)/(len(val_1.index)+len(val_2.index))}")
    """

s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
s_29
s_30
s_31
s_32
s_33
s_34
s_35
s_36
s_37
s_38
s_39
s_40
s_41
s_42
s_43
s_44
s_45
s_46
s_47
s_48
s_49
s_50
s_51
s_52
s_53
s_54
s_55
s_56
s_57
s_58
s_59
s_60
s_61
s_62
s_63
s_64
s_65
s_66
s_67
s_68
s_69
s_70
s_71
s_72
s_73
s_74
s_75
s_76
s_77
s_78
s_79
s_80
s_81
s_82
s_83
s_84
s_85
s_86
s_87
s_88
s_89
s_90
s_91
s_92
s_93
s_94
s_95
s_96
s_97
s_98
s_99
s_100
s_101
s_102
s_103
s_104
s_105
s_106
s_107
s_108
s_109
s_110
s_111
s_112
s_113
s_114
s_115
s_116
s_117
s_118
s_119
s_120
s_121
s_122
s_123
s_124
s_125
s_126
s_127
s_128
s_129
s_130
s_131
s_132
s_133
s_134
s_135
s_136
s_137
s_138
s_139
s_140
s_141
s_142
s_143
s_144
s_145
s_146
s_147
s_148
s_149
s_150
s_151
s_152
s_153
s_154
s_155
s_156
s_157
s_158
s_159
s_160
s_161
s_162
s_163
s_164
s_165
s_166
s_167
s_168
s_169
s_170
s_171
s_172
s_173
s_174
s_175
s_176
s_177
s_178
s_179
s_180
s_181
s_182
s_183
s_184


In [53]:
i_contraception = 0
i_pill = 0
m = tf.keras.metrics.CosineSimilarity(axis=0)
for i, emb in enumerate(embeddings_1):
    last_word = emb[2].split()
    if last_word[1].startswith("murder"):
        i_contraception = i
    if last_word[1].startswith("abortion"):
        i_pill = i
    if i_pill != 0 and i_contraception != 0:
        break

m.update_state(embeddings_1[i_contraception][0], embeddings_1[i_pill][0])
print(f"cos dist {embeddings_1[i_contraception][2]} + {embeddings_1[i_pill][2]}: {m.result().numpy()}")

cos dist is murder + if abortion: 0.3986886143684387


In [ ]:
0.33423537015914917, no people : when someone
----------
0.20209284126758575, pro choice : life question
----------
0.20202544331550598, want anybody : the woman
----------
ctn: 1
s_0
s_1
s_2
s_3
ctn: 2
s_0
s_1
ctn: 3
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
s_29
s_30
s_31
s_32
s_33
s_34
s_35
s_36
s_37
s_38
s_39
s_40
s_41
ctn: 4
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 5
s_0
s_1
s_2
s_3
ctn: 6
s_0
s_1
s_2
s_3
ctn: 7
s_0
s_1
s_2
s_3
ctn: 8
s_0
s_1
ctn: 9
s_0
ctn: 10
s_0
s_1
s_2
ctn: 11
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
ctn: 12
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
ctn: 13
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
ctn: 14
s_0
s_1
s_2
ctn: 15
s_0
s_1
ctn: 16
s_0
s_1
s_2
s_3
s_4
ctn: 17
s_0
s_1
ctn: 18
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
ctn: 19
s_0
s_1
s_2
ctn: 20
s_0
ctn: 21
s_0
s_1
s_2
ctn: 22
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
ctn: 23
s_0
s_1
ctn: 24
s_0
s_1
s_2
s_3
ctn: 25
s_0
ctn: 26
s_0
ctn: 27
s_0
s_1
s_2
ctn: 28
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
ctn: 29
s_0
s_1
ctn: 30
s_0
s_1
s_2
s_3
ctn: 31
s_0
s_1
s_2
ctn: 32
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
ctn: 33
s_0
s_1
ctn: 34
s_0
s_1
s_2
s_3
s_4
s_5
s_6
ctn: 35
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
ctn: 36
s_0
s_1
ctn: 37
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
ctn: 38
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
ctn: 39
s_0
ctn: 40
s_0
s_1
ctn: 41
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
ctn: 42
s_0
s_1
s_2
s_3
ctn: 43
s_0
s_1
ctn: 44
s_0
s_1
ctn: 45
s_0
s_1
s_2
s_3
s_4
ctn: 46
s_0
s_1
s_2
s_3
ctn: 47
s_0
s_1
s_2
s_3
ctn: 48
s_0
s_1
ctn: 49
s_0
ctn: 50
s_0
s_1
s_2
ctn: 51
s_0
s_1
ctn: 52
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
ctn: 53
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
ctn: 54
s_0
s_1
s_2
ctn: 55
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
ctn: 56
s_0
s_1
ctn: 57
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
0.20006251335144043, the criteria : and criminals
----------
0.2002699375152588, for murder : and criminals
----------
0.2004309743642807, the killing : and lots
----------
0.20055560767650604, a sane : and criminals
----------
0.20075766742229462, sane person : and criminals
----------
0.20093530416488647, with intent : and criminals
----------
0.20108762383460999, intent malice : and criminals
----------
0.20122313499450684, malice aforethought : and criminals
----------
0.20138324797153473, prior intention : and criminals
----------
0.20158880949020386, particular victim : and criminals
----------
0.20173104107379913, or anyone : and criminals
----------
0.20190316438674927, the way : and criminals
----------
0.202083557844162, legal excuse : and criminals
----------
0.20230016112327576, or authority : and criminals
----------
0.20245686173439026, clear circumstances : and criminals
----------
0.2026599943637848, degree murder : and criminals
----------
0.20272254943847656, many states : and criminals
----------
0.20292139053344727, a killing : and criminals
----------
0.20307035744190216, torture movement : and criminals
----------
0.20329295098781586, the person : and criminals
----------
0.20350408554077148, the killing : and criminals
----------
0.20368734002113342, the death : and criminals
----------
0.20388482511043549, police officer : and criminals
----------
0.2040196806192398, or prison : and criminals
----------
0.20417603850364685, prison guard : and criminals
----------
0.20432890951633453, an incident : and criminals
----------
0.20454159379005432, another crime : and criminals
----------
0.20467330515384674, or rape : and criminals
----------
0.20486226677894592, degree murder : and criminals
----------
0.20505033433437347, without premeditation : and criminals
----------
0.20524001121520996, with malice : and criminals
----------
0.20534701645374298, second degree : and criminals
----------
0.20550322532653809, degree murder : and criminals
----------
0.20571206510066986, without premeditation : and criminals
----------
0.20583106577396393, the heat : with hate
----------
0.2060202807188034, of passion : and criminals
----------
0.20620323717594147, sudden quarrel : and criminals
----------
0.20638397336006165, or fight : and criminals
----------
0.20655538141727448, degree murder : and criminals
----------
0.20674973726272583, a death : and criminals
----------
0.2068725973367691, reckless lack : and criminals
----------
0.2070625275373459, of concern : and criminals
----------
0.207257941365242, the life : and criminals
----------
0.20743459463119507, of others : and criminals
----------
0.20761257410049438, a gun : with hate
----------
0.20781069993972778, a crowd : and criminals
----------
0.2079443335533142, bashing someone : and criminals
----------
0.20813095569610596, deadly weapon : and criminals
----------
0.2083124965429306, the circumstances : are people
----------
0.20842064917087555, and state : and criminals
----------
0.2086000144481659, state laws : and criminals
----------
0.20870836079120636, laws murder : and criminals
----------
0.20887820422649384, second degree : and criminals
----------
0.20910193026065826, a person : and criminals
----------
0.20930294692516327, a crime : and criminals
----------
0.20952337980270386, a partner : and criminals
----------
0.20971277356147766, the killing : and criminals
----------
0.20984429121017456, or someone : and criminals
----------
0.21002046763896942, the result : and criminals
----------
0.210234135389328, the crime : and criminals
----------
0.21035556495189667, a liquor : is murder
----------
0.21045804023742676, liquor store : and criminals
----------
0.21053236722946167, store stick-up : illigal try
----------
0.21063290536403656, the clerk : the people
----------
0.2107679694890976, hold-up man : and criminals
----------
0.21093837916851044, a bystander : and criminals
----------
0.21103352308273315, armed robber : and criminals
----------
0.21114493906497955, second degree : and criminals
----------
0.21130792796611786, degree murder : and criminals
----------
0.21145908534526825, a charge : with hate
----------
0.21164797246456146, of murder : and criminals
----------
0.21181784570217133, the victim : and criminals
----------
0.21203260123729706, a year : and criminals
----------
0.21224254369735718, the attack : and criminals
----------
0.21245858073234558, unborn child : and criminals
----------
0.2126442939043045, ( fetus : and criminals
----------
0.21282969415187836, be murder : and criminals
----------
0.21302995085716248, was premeditation : and criminals
----------
0.2131672501564026, premeditation malice : and criminals
----------
0.21337030827999115, legal authority : and criminals
----------
0.21353700757026672, thus abortion : with hate
----------
0.21376200020313263, the law : and criminals
----------
0.21385450661182404, : jack : and lots
----------
0.21396863460540771, jack violent : with hate
----------
0.2140548974275589, pregnant girlfriend : of abortions
----------
0.21428713202476501, the fetus : and criminals
----------
0.2143651247024536, involuntary lacks : and criminals
----------
0.21451787650585175, the element : and criminals
----------
0.21470046043395996, of malice : and criminals
----------
0.21480940282344818, malice aforethought : and criminals
----------
ctn: 58
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
0.21490348875522614, all right : and criminals
----------
0.2150871604681015, a fetus : are people
----------
0.21523384749889374, a bunch : and criminals
----------
0.21542593836784363, of cells : and criminals
----------
0.2155652791261673, feels nothing : and criminals
----------
0.2157457321882248, until birth : and criminals
----------
0.21589964628219604, feels everything : and criminals
----------
0.21605755388736725, other people : and criminals
----------
0.21628643572330475, for people : and criminals
----------
0.21647517383098602, religious morals : and criminals
----------
0.21666638553142548, religious morals : and criminals
----------
0.21678906679153442, firstly people : and criminals
----------
0.21695096790790558, the time : and criminals
----------
0.21707187592983246, time fail : and criminals
----------
0.21709612011909485, into account : with hate
----------
0.21722857654094696, the number : and criminals
----------
0.2173970490694046, of women : and criminals
----------
0.2176186442375183, the child : and criminals
----------
0.21783363819122314, the finances : and criminals
----------
0.2180340588092804, her child : and criminals
----------
0.21821902692317963, absolute death : and criminals
----------
0.21845024824142456, that child : and criminals
----------
0.218636155128479, of hand-me-downs : and criminals
----------
0.218819260597229, distant cousins : and criminals
----------
0.21889065206050873, two dollar : the people
----------
0.2190413624048233, dollar shirts : and criminals
----------
0.2192372828722, the salvation : and criminals
----------
0.2193959653377533, salvation army : and criminals
----------
0.2196003496646881, the child : and criminals
----------
0.21980181336402893, no meals : and criminals
----------
0.21994177997112274, for lunch : of abortions
----------
0.2201046496629715, on weekdays : and criminals
----------
0.2202574610710144, at school : and criminals
----------
0.22041425108909607, free/reduced lunch : the people
----------
0.22053679823875427, lunch agreement : are people
----------
0.22064515948295593, some schools : and criminals
----------
0.22085292637348175, actual scenario : and criminals
----------
0.22106914222240448, this child : and criminals
----------
0.22127486765384674, in school : and criminals
----------
0.2214948534965515, the duration : and criminals
----------
0.22166459262371063, of his/her : and criminals
----------
0.22188955545425415, his/her life : and criminals
----------
0.22210407257080078, that child : and criminals
----------
0.22230671346187592, entire life : and criminals
----------
0.22246593236923218, life he/she : and criminals
----------
0.22265061736106873, the reason : and criminals
----------
0.2228299230337143, tiny apartment : and criminals
----------
0.22301718592643738, of welfare : and criminals
----------
0.22305725514888763, said please : illigal try
----------
0.2230909764766693, please explain : are people
----------
0.22332939505577087, a child : and criminals
----------
0.223502978682518, have nothing : and criminals
----------
0.22370734810829163, guilty conscience : and criminals
----------
0.2238539308309555, over something : and criminals
----------
0.22404853999614716, his/her fault : and criminals
----------
0.22415104508399963, getting rid : and criminals
----------
0.22434914112091064, of lives : and criminals
----------
0.22455286979675293, the situation : and criminals
----------
0.22473472356796265, stereotypical woman : and criminals
----------
0.22495552897453308, her child : and criminals
----------
0.22513197362422943, out asap : and criminals
----------
0.22528807818889618, yes birth : and criminals
----------
0.22545985877513885, birth control : and criminals
----------
0.22561341524124146, best rebuttal : with hate
----------
0.22576279938220978, shoulda woulda : and criminals
----------
0.22590206563472748, woulda coulda : and criminals
----------
0.2261001020669937, a woman : and criminals
----------
0.22629626095294952, from society : and criminals
----------
0.22650811076164246, her child : and criminals
----------
0.22670947015285492, illegal abortions : and criminals
----------
0.2269202470779419, accelerated rate : and criminals
----------
0.22714099287986755, the child : and criminals
----------
0.22739017009735107, the woman : and criminals
----------
0.22757862508296967, main point : and criminals
----------
0.22760792076587677, : give : her self
----------
0.22778041660785675, for adoption : and criminals
----------
0.22795754671096802, adoption scenario : and criminals
----------
0.22808411717414856, do something : and criminals
----------
0.2282324880361557, ) scenario : and criminals
----------
0.22843125462532043, : children : and criminals
----------
0.22859516739845276, in adoption : and criminals
----------
0.22878926992416382, adoption centers : and criminals
----------
0.228989839553833, horrible thing : and criminals
----------
0.2291659712791443, sunshine-y rainbow : and criminals
----------
0.22927604615688324, unicorn glitterfest : and criminals
----------
0.22941240668296814, farthest things : and criminals
----------
0.2295888066291809, adopting children : and criminals
----------
0.22975632548332214, at adoption : and criminals
----------
0.22994616627693176, adoption agencies : and criminals
----------
0.23010699450969696, a pet : and criminals
----------
0.23028071224689484, pet store : and criminals
----------
0.23043370246887207, store cutest : and criminals
----------
0.2306239753961563, youngest ones : and criminals
----------
0.23079632222652435, this kid : and criminals
----------
0.2309676706790924, cutest thing : and criminals
----------
0.2311633974313736, the world : and criminals
----------
0.23130357265472412, some kind : and criminals
----------
0.23151454329490662, super talent : and criminals
----------
0.2316565364599228, talent his/her : and criminals
----------
0.2318481057882309, his/her chances : and criminals
----------
0.23203539848327637, of adoption : and criminals
----------
0.23225358128547668, poor kid : and criminals
----------
0.2324105054140091, entire childhood : and criminals
----------
0.2325478494167328, a foster : and criminals
----------
0.23272983729839325, foster home : and criminals
----------
0.2328948974609375, from home : and criminals
----------
0.2330782562494278, to home : and criminals
----------
0.23325717449188232, fair situation : and criminals
----------
0.2334970235824585, the child : and criminals
----------
0.23370134830474854, oppositional point : and criminals
----------
0.2338884323835373, the mother : and criminals
----------
0.23406195640563965, steady relationship : and criminals
----------
0.23428095877170563, her partner : and criminals
----------
0.23449496924877167, a child : and criminals
----------
0.23460477590560913, physical and/or : and criminals
----------
0.23475907742977142, genetic issues : and criminals
----------
0.23495551943778992, this mother : and criminals
----------
0.23514340817928314, give birth : and criminals
----------
0.2353244423866272, all families : and criminals
----------
0.23548373579978943, have millions : and criminals
----------
0.23566238582134247, of dollars : and criminals
----------
0.23586009442806244, these issues : and criminals
----------
0.23602785170078278, these families : and criminals
----------
0.2361888438463211, families savings : and criminals
----------
0.23634053766727448, savings dry : and criminals
----------
0.23653282225131989, any money : and criminals
----------
0.23674708604812622, a child : and criminals
----------
0.23684871196746826, because something : and criminals
----------
0.23702989518642426, with him/her : and criminals
----------
0.23722821474075317, a child : and criminals
----------
0.2374061942100525, his/her entrance : and criminals
----------
0.2376081943511963, this world : and criminals
----------
0.23778824508190155, that mother : and criminals
----------
0.23798254132270813, give birth : and criminals
----------
0.23818014562129974, more pain : and criminals
----------
0.23842860758304596, that child : and criminals
----------
0.23857331275939941, for years : and criminals
----------
0.23879462480545044, a mother : and criminals
----------
0.23901617527008057, her pregnancy : and criminals
----------
0.23916959762573242, save him/her : and criminals
----------
0.23940254747867584, that illness : and criminals
----------
0.23951661586761475, in conclusion : are people
----------
0.23968937993049622, no right : and criminals
----------
ctn: 59
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
0.23983456194400787, a sperm : of abortions
----------
0.24004624783992767, a fetus : with hate
----------
0.240248903632164, its ment : and criminals
----------
0.24044351279735565, be miscarriages : and criminals
----------
0.24061338603496552, or positives : and criminals
----------
0.2408161759376526, the negatives : and criminals
----------
0.24101240932941437, your perspective : and criminals
----------
0.24116608500480652, convincing argument : and criminals
----------
0.24132059514522552, both sides : and criminals
----------
0.24152320623397827, define life : and criminals
----------
0.2416498363018036, gods actions : and criminals
----------
0.24185025691986084, pro choice : and criminals
----------
0.24201661348342896, am pro-choice : and criminals
----------
0.24217209219932556, an abortion : with hate
----------
0.2423141598701477, all means : and criminals
----------
0.24254688620567322, my choice : and criminals
----------
0.24267597496509552, the death : with hate
----------
0.2427632063627243, that god : and criminals
----------
0.24293303489685059, old testament : with hate
----------
0.24311110377311707, its note : and criminals
----------
0.24329973757266998, pro life : and criminals
----------
0.24349454045295715, an atheist : and criminals
----------
0.24363017082214355, to emotions : and criminals
----------
0.24381034076213837, the spirit : are people
----------
0.24400697648525238, a belief : and criminals
----------
ctn: 60
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
0.24420106410980225, just abortion : and criminals
----------
0.2443431168794632, which side : are people
----------
0.24447743594646454, a break : of abortions
----------
0.24467509984970093, multiple aspects : and criminals
----------
0.24485747516155243, the definition : are people
----------
0.24501372873783112, debatable subjects : and criminals
----------
0.24515753984451294, abortion euthanasia : and criminals
----------
0.24534811079502106, the death : and criminals
----------
0.2455059289932251, death penalty : and criminals
----------
0.24565812945365906, penalty war : and criminals
----------
0.24584488570690155, these things : and criminals
----------
0.24603019654750824, include someone : and criminals
----------
0.24617841839790344, what pro-lifers : and criminals
----------
0.24633143842220306, different topics : and criminals
----------
0.24648945033550262, into conflicts : and criminals
----------
0.2466241866350174, within ourselves : and criminals
----------
0.24663840234279633, for instance : illigal try
----------
0.24671900272369385, instance war : and criminals
----------
0.2468762844800949, support war : and criminals
----------
0.2470729947090149, support people : and criminals
----------
0.24724695086479187, these people : and criminals
----------
0.2474527359008789, the military : and criminals
----------
0.24760794639587402, the knowledge : and criminals
----------
0.2478029578924179, their country : and criminals
----------
0.2479642927646637, our soldiers : and criminals
----------
0.248160719871521, their families : and criminals
----------
0.24833497405052185, their friends : and criminals
----------
0.24852254986763, pro choice : and criminals
----------
0.24872691929340363, their family : and criminals
----------
0.24888762831687927, family member/ : and criminals
----------
0.24903604388237, member/ friend : and criminals
----------
0.24918290972709656, support death : and criminals
----------
0.24935796856880188, one dyings : and criminals
----------
0.24947084486484528, i dont : and criminals
----------
0.24964386224746704, a supporter : and criminals
----------
0.24980486929416656, of life : and criminals
----------
0.24996864795684814, no matter : and criminals
----------
0.25004059076309204, no exceptions : of abortions
----------
0.2501406967639923, no way : and criminals
----------
0.2502981126308441, no way : and criminals
----------
0.25045087933540344, any war : and criminals
----------
0.25054362416267395, your life : my gramps
----------
0.2505641281604767, self education : and criminals
----------
0.2506783604621887, and consistency : and criminals
----------
0.2507788836956024, either side : are people
----------
0.25089046359062195, pro choice : are people
----------
0.2509932816028595, pro life : and criminals
----------
0.2511444389820099, the topics : with hate
----------
0.2511872947216034, at hand : with hate
----------
0.25125786662101746, the abortion : of abortions
----------
0.2513132095336914, abortion topic : are people
----------
0.2515280246734619, the womans : and criminals
----------
0.2516805827617645, womans choice : and criminals
----------
0.25190767645835876, this soldiers : and criminals
----------
0.2520958483219147, soldiers choice : and criminals
----------
0.25210660696029663, hang ribbons : kill fetises
----------
0.252151221036911, this though : this girl
----------
0.25226807594299316, our soldiers : and criminals
----------
0.2523419260978699, an example : and criminals
----------
0.252397358417511, or punctuation : of abortions
----------
ctn: 61
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.2524096965789795, no way : a stance
----------
0.2524668276309967, way pro-abortion : and criminals
----------
0.2526015341281891, against abortion : and criminals
----------
0.25265827775001526, are cases : are people
----------
0.25272539258003235, which abortion : of abortions
----------
0.2527448534965515, for example : in austin
----------
0.2528603971004486, a woman : a coat
----------
0.25298765301704407, for someone : and criminals
----------
0.25311434268951416, their baby : of abortions
----------
0.25320228934288025, the dna : with hate
----------
0.2533535957336426, of someone : and criminals
----------
0.25345298647880554, rape someone : and criminals
----------
0.2535775899887085, this baby : the people
----------
0.25367021560668945, permanant reminder : the cause
----------
0.25375983119010925, horrible night : the cause
----------
0.2539418041706085, the mother : and criminals
----------
0.2541503608226776, the baby : are people
----------
0.2542266845703125, another point : a question
----------
0.2542977035045624, my dad : her mie
----------
0.2544182240962982, is abortion : with hate
----------
0.2544752061367035, 24 weeks : are people
----------
0.25455567240715027, a photo : the cause
----------
0.2546832263469696, a baby : of people
----------
0.25479641556739807, 24 weeks : are people
----------
0.2549172639846802, a baby : of abortions
----------
0.25496646761894226, beginning stages : are people
----------
0.2551683187484741, the pregnancy : are people
----------
0.2552715539932251, proper reason : and lots
----------
0.2553224265575409, 24 weeks : are people
----------
0.2554040551185608, 12 weeks : are people
----------
ctn: 62
s_0
s_1
s_2
s_3
0.2554504871368408, if everybody : the people
----------
0.25552746653556824, babies-for-profit adoption : of abortions
----------
0.25558263063430786, adoption industry : and criminals
----------
0.2557092308998108, the answer : the cause
----------
0.2559138238430023, integral facility : are people
----------
0.2560747265815735, modern society : and criminals
----------
ctn: 63
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.25607916712760925, for dea : by scientists
----------
0.25610971450805664, dea people : are people
----------
0.2561880052089691, a pulse : a question
----------
0.2562202215194702, a bit : a question
----------
0.25627511739730835, in response : with hate
----------
0.2563244700431824, last statement : a story
----------
0.2564484477043152, unborn children : and criminals
----------
0.256509393453598, same way : my gramps
----------
0.2566426992416382, the killing : the people
----------
0.2567935883998871, of animals : and criminals
----------
0.2569408714771271, provide food : and criminals
----------
0.25703707337379456, the killing : with hate
----------
0.2571752071380615, of animals : and criminals
----------
0.25725245475769043, my life : with hate
----------
0.25740674138069153, inverted commas : and criminals
----------
0.2575252652168274, the unborns : the people
----------
ctn: 64
s_0
s_1
0.25767266750335693, the death : the people
----------
0.25775277614593506, death penalty : are people
----------
0.2578427791595459, the wars : with hate
----------
0.25790026783943176, in iraq : and criminals
----------
0.25798484683036804, and afghanistan : and criminals
----------
ctn: 65
s_0
s_1
0.2579882740974426, as understood : basic books
----------
0.25805869698524475, involuntary manslaughter : and criminals
----------
0.25813427567481995, the killing : the cause
----------
0.2582250237464905, another member : are people
----------
0.2582858204841614, of society : and criminals
----------
0.2583327889442444, without intent : with hate
----------
0.2584666311740875, a miscarriage : of abortions
----------
0.2585890591144562, the fetus : are people
----------
0.2586804926395416, a member : are people
----------
0.2587570250034332, of society : and criminals
----------
ctn: 66
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.258806973695755, with kmb56yuk : with hate
----------
0.2588953375816345, because abortion : and criminals
----------
0.259062260389328, an issue : and criminals
----------
0.25918662548065186, all sides : and criminals
----------
0.2593348026275635, strong feelings : and criminals
----------
0.25948724150657654, and judgement : and criminals
----------
0.25956615805625916, radical desicion : are people
----------
0.2597644031047821, each mother : are people
----------
0.25982362031936646, that women : and criminals
----------
0.25993749499320984, many sides : and criminals
----------
0.2600833773612976, a women : and criminals
----------
0.2601281404495239, women health : are people
----------
0.26022714376449585, big danger : with hate
----------
0.26034143567085266, the future : with hate
----------
0.26048803329467773, a baby : are people
----------
0.2605593204498291, secondly abortion : of abortions
----------
0.26075664162635803, a murder : and criminals
----------
0.26096057891845703, unborn children : and criminals
----------
0.2610233426094055, does life : of abortions
----------
0.26107755303382874, some people : are people
----------
0.26117071509361267, the embryo : the cause
----------
0.26124051213264465, in womb : of abortions
----------
0.2612963318824768, innoticible border : the pre-embryo
----------
0.261373370885849, when fetus : of abortions
----------
0.2615453600883484, a person : are people
----------
0.261691689491272, by law : and criminals
----------
0.2617531418800354, equal right : are people
----------
0.2619544565677643, for life : and criminals
----------
0.2620583474636078, a action : with hate
----------
0.26212140917778015, prove women : and criminals
----------
0.2621711194515228, women rights : and criminals
----------
0.26232269406318665, and freedom : and criminals
----------
0.2625505328178406, own bodies : and criminals
----------
0.26274076104164124, other person : and criminals
----------
0.26285669207572937, person body : and criminals
----------
0.26297757029533386, every person : are people
----------
0.26310765743255615, for life : with hate
----------
0.2631933391094208, all women : and criminals
----------
0.2633042335510254, a pregnancy : are people
----------
0.2634298503398895, by abortion : with hate
----------
0.2635291814804077, use practise : with hate
----------
0.2636602818965912, after-effect beforehand : and criminals
----------
ctn: 67
s_0
s_1
s_2
s_3
0.26372939348220825, no one : are people
----------
0.26384320855140686, a woman : with hate
----------
0.2639928460121155, the child : are people
----------
0.2641880214214325, the child : with hate
----------
0.26440924406051636, her choice : and criminals
----------
0.2645253837108612, the mother : the people
----------
0.26468726992607117, the child : are people
----------
0.2648354470729828, a life : are people
----------
0.2649652659893036, of poverty : and criminals
----------
0.2650282680988312, pregnant mothers : are people
----------
0.26512396335601807, mothers shoes : are people
----------
ctn: 68
s_0
s_1
0.26518917083740234, a medal : a question
----------
0.2652221620082855, suicide assistance : are people
----------
0.2653021216392517, by doctors : and criminals
----------
ctn: 69
s_0
0.26538702845573425, the government : the people
----------
0.26546692848205566, no room : with hate
----------
0.2656007707118988, a woman : and criminals
----------
0.26575350761413574, their body : and criminals
----------
ctn: 70
s_0
ctn: 71
s_0
s_1
s_2
s_3
s_4
0.2657889127731323, the right : them mammals
----------
0.2658923268318176, make decisions : with hate
----------
0.2660815119743347, own bodies : and criminals
----------
0.2661643624305725, to viability : of abortions
----------
0.26627036929130554, have rights : with hate
----------
0.2663887143135071, the fetus : the cause
----------
0.2664707899093628, have rights : with hate
----------
0.26667919754981995, the woman : and criminals
----------
0.2667904198169708, woman decision : are people
----------
0.2669147253036499, an abortion : with hate
----------
0.2669680416584015, takes precedence : are people
----------
0.26714545488357544, the interests : and criminals
----------
0.26742175221443176, the fetus : and criminals
----------
0.2675258219242096, in america : with hate
----------
0.267568975687027, federal abortion : of abortions
----------
0.2676330506801605, abortion amendment : are people
----------
ctn: 72
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
0.2677982747554779, the mother : the people
----------
0.2679558992385864, mother baby : are people
----------
0.26819533109664917, her decision : and criminals
----------
0.26834455132484436, the baby : of abortions
----------
0.26846563816070557, her body : a coat
----------
0.26857122778892517, her body : a coat
----------
0.2686680555343628, her body : an infection
----------
0.2687714993953705, her choice : with hate
----------
0.2688613831996918, her baby : an abortion
----------
0.26895663142204285, her choice : the cause
----------
0.2690141797065735, does life : make abortions
----------
0.2691269516944885, another debate : and criminals
----------
0.2692459225654602, different definitions : the cause
----------
0.2692625820636749, equal ground : proper protection
----------
0.269363135099411, say life : and criminals
----------
0.2693996727466583, life begins : are people
----------
0.2695731520652771, the baby : the people
----------
0.2697795331478119, the womb : and criminals
----------
ctn: 73
s_0
s_1
s_2
s_3
s_4
s_5
0.2698187232017517, have arms : of abortions
----------
0.26986315846443176, and legs : and criminals
----------
0.26992881298065186, and hair : and criminals
----------
0.26998138427734375, and fingernails : and criminals
----------
0.2700065076351166, human dna : coat hanger
----------
0.27003684639930725, dna skin : an infection
----------
0.2701203525066376, skin cells : and criminals
----------
0.27013224363327026, human dna : of infection
----------
0.2703206241130829, a parasite : and criminals
----------
0.2704903483390808, a woman : and criminals
----------
0.27063843607902527, the woman : are people
----------
0.27080780267715454, her womb : are people
----------
0.2710193395614624, the fetus : are people
----------
ctn: 74
s_0
s_1
s_2
0.2711644768714905, an individual : and criminals
----------
0.2712498605251312, a law : the cause
----------
0.27141472697257996, a woman : are people
----------
0.2715972661972046, a hysterectomy : are people
----------
0.2716963589191437, her tubes : of abortions
----------
ctn: 75
s_0
s_1
s_2
0.271777868270874, any statistics : the cause
----------
0.2718398869037628, suicide alcoholism : are people
----------
0.2718811333179474, and drug : and criminals
----------
0.27196162939071655, drug addiction : and criminals
----------
0.27199432253837585, direct result : the cause
----------
0.27210333943367004, an abortion : the people
----------
0.27224603295326233, your argument : with hate
----------
ctn: 76
s_0
s_1
ctn: 77
s_0
s_1
0.27235808968544006, for criminalization : and criminals
----------
0.27245914936065674, the lay : with hate
----------
0.272589772939682, the street : and criminals
----------
0.27277007699012756, no poetry : and criminals
----------
0.27291086316108704, or metaphors : and criminals
----------
0.27295199036598206, how communists : and criminals
----------
0.27297699451446533, before capitalists : and criminals
----------
0.27300652861595154, in wwii : with hate
----------
ctn: 78
s_0
s_1
s_2
s_3
s_4
0.27314293384552, about citizens : and criminals
----------
0.27317720651626587, all persons : are people
----------
0.2732524871826172, united states : are people
----------
0.2733810842037201, the jurisdiction : are people
----------
0.2734255790710449, jurisdiction thereof : are people
----------
0.2735271155834198, are citizens : and criminals
----------
0.27359986305236816, united states : are people
----------
0.2737025022506714, the state : the people
----------
0.2737114131450653, state wherein : all circumstances
----------
0.2737698554992676, no state : are people
----------
0.2738872468471527, any law : and criminals
----------
0.27399134635925293, the privileges : with hate
----------
0.2740382254123688, or immunities : and lots
----------
0.2741520404815674, of citizens : and criminals
----------
0.27421727776527405, united states : are people
----------
0.27426594495773315, any state : are people
----------
0.27438628673553467, any person : and criminals
----------
0.27453404664993286, of life : and criminals
----------
0.2746509313583374, life liberty : and criminals
----------
0.2747783064842224, or property : and criminals
----------
0.27487799525260925, due process : and criminals
----------
0.2749854624271393, of law : and criminals
----------
0.2749951481819153, nor deny : are people
----------
0.27505743503570557, any person : and criminals
----------
0.27510932087898254, its jurisdiction : the world
----------
0.2751675844192505, equal protection : with hate
----------
0.27533143758773804, any person : and criminals
----------
0.2755182087421417, of life : and criminals
----------
0.2757037580013275, this country : and criminals
----------
0.2757700979709625, says citizens : and criminals
----------
ctn: 79
s_0
s_1
s_2
s_3
0.2757769823074341, to state : least use
----------
0.2759150564670563, committing abortion : and criminals
----------
0.2760845422744751, committing murder : and criminals
----------
0.2762652635574341, another person : and criminals
----------
0.2763579487800598, person right : and criminals
----------
0.2765316367149353, to life : and criminals
----------
0.2766115665435791, my belief : with hate
----------
0.27670952677726746, non-religious evidence : with hate
----------
0.27679067850112915, that abortion : of abortions
----------
0.27692973613739014, as murder : and criminals
----------
0.27707332372665405, any government : and criminals
----------
0.27716097235679626, separate religion : and criminals
----------
0.2772729694843292, and law : and criminals
----------
0.27735427021980286, no government : are people
----------
0.27744027972221375, separates religion : with hate
----------
0.2775525748729706, and law : and criminals
----------
0.2776133418083191, the right : with hate
----------
0.27766820788383484, its citizens : are people
----------
ctn: 80
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.2777066230773926, which way : in austin
----------
0.2777346074581146, that effect : the facts
----------
0.27788710594177246, that child : are people
----------
0.278012752532959, your mouth : with hate
----------
0.27814772725105286, your opinions : and criminals
----------
0.27826547622680664, their stories : the cause
----------
0.27832579612731934, this statistic : with hate
----------
0.2783723473548889, the majority : the people
----------
0.27846434712409973, of abortions : with hate
----------
0.27850762009620667, for reasons : with hate
----------
0.27861228585243225, of convenience : and criminals
----------
0.2786954641342163, a questionnaire : a question
----------
0.2787298262119293, questionnaire poll : coat hanger
----------
0.2788507044315338, an abortion : with hate
----------
0.2789209485054016, abortion cause : are people
----------
0.27908816933631897, this information : with hate
----------
0.27922701835632324, your argument : with hate
----------
0.2793343663215637, this point : with hate
----------
0.27933764457702637, it pure : and mccormick
----------
0.2793835699558258, on opinion : and criminals
----------
ctn: 81
s_0
s_1
s_2
s_3
0.27944037318229675, great point : a story
----------
0.27953505516052246, a convince : a question
----------
0.2795429825782776, cant rebuttel : kennedy inst
----------
0.2796480059623718, a convince : a question
----------
0.27966806292533875, cant think : are people
----------
0.2797069549560547, of anything : with hate
----------
0.2797425091266632, good right : illigal try
----------
0.27979931235313416, great point : a story
----------
ctn: 82
s_0
s_1
s_2
s_3
s_4
0.2798442542552948, many reasons : the cause
----------
0.27994781732559204, for abortion : with hate
----------
0.2801031470298767, than adoption : and criminals
----------
0.2801331877708435, emotional scars : and criminals
----------
0.2802845537662506, the infant : are people
----------
0.2803763151168823, for adoption : with hate
----------
0.2804994285106659, the probability : are people
----------
0.2806893587112427, the child : and criminals
----------
0.2808249592781067, adopting parent : are people
----------
0.2808970808982849, emotional scars : and criminals
----------
0.2810550630092621, the woman : are people
----------
0.28109505772590637, psychological scars : of abortions
----------
0.2812039852142334, unwanted pregnancy : of abortions
----------
0.28131160140037537, the weight : the cause
----------
0.28134021162986755, weight gain : of abortions
----------
0.2814784348011017, a pregnancy : is murder
----------
0.2815389335155487, of job : with hate
----------
0.28164923191070557, job opportunities : of abortions
----------
0.2816965878009796, opportunities time : illigal try
----------
0.28183528780937195, from work : are people
----------
0.2819115221500397, work changes : coat hanger
----------
0.28208521008491516, the body : of abortions
----------
0.2821321487426758, body chemistry : coat hanger
----------
0.28226175904273987, financial burden : the cause
----------
0.2823825776576996, child birth : and criminals
----------
0.28244704008102417, birth etc : and criminals
----------
ctn: 83
s_0
s_1
s_2
s_3
s_4
s_5
s_6
0.2825731635093689, a woman : are people
----------
0.28258654475212097, pro-life suggests : are people
----------
0.2827327251434326, the child : are people
----------
0.2827780544757843, all women : are people
----------
0.28291141986846924, the child : are people
----------
0.2829870581626892, the conditions : the cause
----------
0.28312188386917114, the woman : the people
----------
0.28330737352371216, the child : are people
----------
0.28342872858047485, first place : and criminals
----------
0.2835993766784668, the child : are people
----------
0.28375571966171265, the process : the people
----------
0.2839008569717407, of birth : and criminals
----------
0.2840423882007599, forcing death : and criminals
----------
0.2841193974018097, a study : a question
----------
0.2842044532299042, donohue-levitt hypothesis : the people
----------
0.28430745005607605, the mother : the people
----------
0.28433987498283386, mother dies : fetus bipedal
----------
0.28449374437332153, the child : the people
----------
0.28468558192253113, the child : and criminals
----------
0.2848939895629883, unloved children : and criminals
----------
0.28499358892440796, in gangs : and criminals
----------
0.28500550985336304, and street : and criminals
----------
0.2850436866283417, street violence : and criminals
----------
0.2851664423942566, away choice : with hate
----------
0.28523480892181396, not choice : are people
----------
0.28535860776901245, and freedom : and criminals
----------
0.2854514718055725, main values : the people
----------
0.2856003940105438, of america : and criminals
----------
ctn: 84
s_0
s_1
s_2
s_3
s_4
s_5
0.2856629490852356, the notion : with hate
----------
0.28575074672698975, anti-choice movement : and criminals
----------
0.28577399253845215, has anything : with hate
----------
0.2859356999397278, with life : and criminals
----------
0.2859503924846649, if anything : because someone
----------
0.28601107001304626, social conservativism : and criminals
----------
0.2860342562198639, behind anti-choice : with hate
----------
0.28618359565734863, and pro-execution : and criminals
----------
0.2863052189350128, an embryo : are people
----------
0.2864827811717987, or fetus : and criminals
----------
0.28656527400016785, left foot : of abortions
----------
0.2866760194301605, without personhood : and criminals
----------
0.2867860198020935, primary rights : and criminals
----------
0.2868095338344574, mere fact : the cause
----------
0.2868991792201996, an embryo : of abortions
----------
0.2869539260864258, the right : the cause
----------
0.28709086775779724, the cost : are people
----------
0.2872925400733948, the woman : and criminals
----------
0.2874091565608978, woman autonomy : are people
----------
0.2874923348426819, for life : and criminals
----------
0.28763261437416077, about quality : and criminals
----------
0.287712037563324, just quantity : with hate
----------
0.28781548142433167, between people : and criminals
----------
0.28795233368873596, potential people : and criminals
----------
ctn: 85
s_0
s_1
s_2
0.2881726920604706, a murder : and criminals
----------
0.2882970869541168, when mother : and criminals
----------
0.28841206431388855, take care : and criminals
----------
0.28859779238700867, provide protection : and criminals
----------
0.28878673911094666, her baby : with hate
----------
0.2889018952846527, a baby : a coat
----------
0.2889813184738159, a world : are people
----------
0.2890380322933197, where none : and criminals
----------
0.2891191840171814, takes care : with hate
----------
0.28922173380851746, one baby : the people
----------
0.28938406705856323, more baby : are people
----------
0.2894657254219055, harsh world : are people
----------
ctn: 86
s_0
s_1
0.2895512282848358, the case : the cause
----------
0.28959640860557556, what rape : of abortions
----------
0.2896651029586792, is life : with hate
----------
0.28966665267944336, and prevalent : and non-self
----------
0.2897195816040039, on earth : with hate
----------
0.2897545099258423, earth everyday : illigal try
----------
0.2898413836956024, the germs : the people
----------
ctn: 87
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
0.2899302840232849, a difference : a question
----------
0.2900455892086029, for god : and criminals
----------
0.29015716910362244, the rights : and criminals
----------
0.29029372334480286, of others : and criminals
----------
0.29036200046539307, if abortion : of abortions
----------
0.2904670834541321, religious standpoint : and criminals
----------
0.29056671261787415, the government : and criminals
----------
0.29073089361190796, religious values : and criminals
----------
0.29091373085975647, on people : and criminals
----------
0.2909198999404907, necessarily share : and criminals
----------
0.29097944498062134, if abortions : of abortions
----------
0.29112204909324646, these people : and criminals
----------
0.291151762008667, and share : and criminals
----------
0.2913978099822998, your values : and criminals
----------
0.291553258895874, illegal abortions : and criminals
----------
0.2916328012943268, both mother : and criminals
----------
0.2917586863040924, and child : and criminals
----------
0.2918907105922699, your god : the cause
----------
0.29197707772254944, the world : are people
----------
0.29208800196647644, the world : and criminals
----------
0.29221388697624207, the opinions : with hate
----------
0.2922978103160858, of others : and criminals
----------
0.2923683822154999, the moment : the cause
----------
0.2924245297908783, stop abortion : of abortions
----------
0.29243770241737366, and tell : and criminals
----------
0.29250097274780273, tell others : and criminals
----------
0.29266470670700073, their morals : and criminals
----------
0.2928380072116852, a line : are people
----------
0.2929689586162567, bigoted ass : and criminals
----------
0.2930060923099518, ( regardless : and criminals
----------
0.2930492162704468, of weather : and criminals
----------
0.29310479760169983, an individual : are people
----------
0.29328590631484985, have abortions : and criminals
----------
0.2934175133705139, your church : and criminals
----------
0.29345646500587463, hold building : with hate
----------
0.293539434671402, young parents : and criminals
----------
0.2935730814933777, to budget : are people
----------
0.2936086356639862, budget health : and criminals
----------
0.2936566472053528, health information : and criminals
----------
0.2937163710594177, information etc : and criminals
----------
0.2937619984149933, low cost : are people
----------
0.29388120770454407, ) child : and criminals
----------
0.2939888834953308, child care : and criminals
----------
0.29406389594078064, counselling etc : and criminals
----------
0.2941388487815857, this way : the cause
----------
0.29417696595191956, doing something : of abortions
----------
0.29430878162384033, a sign : the cause
----------
0.2943330705165863, giving people : are people
----------
0.29439350962638855, bad impressions : the cause
----------
0.2944830060005188, support people : are people
----------
0.2946428656578064, have abortions : and criminals
----------
0.2947872281074524, support people : and criminals
----------
0.2948306202888489, and thats : and criminals
----------
0.29495593905448914, what jesus : and criminals
----------
0.29506438970565796, improved sex : with hate
----------
0.29525330662727356, that pregnancies : and criminals
----------
0.2954173684120178, that abortions : and criminals
----------
0.2954840064048767, be members : and criminals
----------
0.2955753207206726, in abortion : with hate
----------
0.29565417766571045, abortion clinics : and criminals
----------
0.29580551385879517, the girls : and criminals
----------
0.29605966806411743, the procedure : and criminals
----------
0.2960778772830963, its tell : our society
----------
0.2961457073688507, tell people : and criminals
----------
0.2962959110736847, your religion : and criminals
----------
0.296434223651886, your values : and criminals
----------
0.29647666215896606, down others : and criminals
----------
0.2965531349182129, others throats : and criminals
----------
0.29656943678855896, throats thats : and criminals
----------
0.2966383397579193, putting religion : and criminals
----------
0.2967313528060913, in government : and criminals
----------
0.29688209295272827, for people : and criminals
----------
0.29700997471809387, the religion : with hate
----------
0.29715433716773987, for people : and criminals
----------
0.2972232699394226, hindu government : and criminals
----------
0.29736095666885376, your morals : and criminals
----------
0.2974345088005066, and worldview : and criminals
----------
0.29755812883377075, your rights : and criminals
----------
ctn: 88
s_0
s_1
0.29762640595436096, if abortion : of abortions
----------
ctn: 89
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.29767465591430664, your reply : an infection
----------
0.2977127730846405, interested parties : are people
----------
0.29784074425697327, the mother : are people
----------
0.2979809641838074, a petition : are people
----------
0.2981208860874176, the court : are people
----------
0.29828524589538574, the problem : and criminals
----------
0.29836586117744446, the court : the government
----------
0.2984045743942261, ad litem : coat hanger
----------
0.2984479069709778, on behalf : and lots
----------
0.29864293336868286, unborn child : and criminals
----------
0.2987741231918335, ( fetus : are people
----------
0.29882627725601196, interested parties : are people
----------
0.2989087998867035, the circumstances : the people
----------
0.29900500178337097, the court : are people
----------
0.29914623498916626, their findings : of abortions
----------
0.29923027753829956, the defense : the people
----------
0.29945680499076843, the child : and criminals
----------
0.29953405261039734, child welfare : and criminals
----------
0.29966026544570923, unwed moms : and criminals
----------
0.2997722625732422, prospective dad : are people
----------
0.29985323548316956, the picture : are people
----------
0.2999744415283203, that matter : and criminals
----------
0.30002546310424805, those cases : the government
----------
0.3000921308994293, the court : the government
----------
0.3001023828983307, in fact : in austin
----------
0.30026325583457947, the law : are people
----------
0.30042895674705505, the mom : are people
----------
0.30054429173469543, the right : with hate
----------
0.30071231722831726, the choice : with hate
----------
0.3009660243988037, the child : and criminals
----------
0.3011004328727722, ( fetus : are people
----------
0.30119818449020386, condone abortion : and criminals
----------
0.3013542592525482, because legality : and criminals
----------
0.3014982044696808, of abortion : and criminals
----------
0.30168890953063965, the law : and criminals
----------
0.3018812835216522, the land : and criminals
----------
0.3019584119319916, a ballot : a question
----------
0.30203408002853394, the court : the people
----------
0.30205291509628296, court rules : life begins
----------
0.3021819293498993, the issue : are people
----------
0.30224907398223877, an opinion : and criminals
----------
ctn: 90
s_0
s_1
0.3023720383644104, a person : are people
----------
0.30244776606559753, a right : the cause
----------
0.3026258051395416, their body : and criminals
----------
0.3026943504810333, without others : and criminals
----------
ctn: 91
s_0
s_1
s_2
0.3027661144733429, fine example : the cause
----------
0.30279627442359924, of reductio : and criminals
----------
0.30286917090415955, ad absurdum : and criminals
----------
0.30299657583236694, the fetus : the cause
----------
0.30312031507492065, a member : are people
----------
0.3032394051551819, of society : and criminals
----------
0.3033677935600281, to rights : and criminals
----------
0.30352121591567993, and protections : and criminals
----------
0.3037388026714325, the application : are people
----------
0.303887277841568, said rights : and criminals
----------
0.3039286434650421, far-out examples : the causes
----------
0.30402109026908875, recognize fetuses : of abortions
----------
0.30410638451576233, full members : are people
----------
0.3042941391468048, of society : and criminals
----------
0.30431225895881653, society hence : are people
----------
0.30447718501091003, the abortion : with hate
----------
0.30461642146110535, is murder : and criminals
----------
0.3047085106372833, murder crowd : and criminals
----------
0.3048439621925354, are hypocrites : and criminals
----------
ctn: 92
s_0
s_1
s_2
0.3048986792564392, the alternative : the cause
----------
0.30504274368286133, unwanted child : and criminals
----------
0.30507099628448486, economic study : a question
----------
0.3050806224346161, in areas : in mammals
----------
0.3051348924636841, where abortion : of abortions
----------
0.3051970303058624, 20 years : of abortions
----------
0.3052453398704529, years crime : and criminals
----------
0.3052738308906555, crime rates : and criminals
----------
0.30529388785362244, not proof : not abortion
----------
0.30535978078842163, direct cause : and criminals
----------
0.3054712116718292, & effect : and criminals
----------
0.3055167496204376, the number : the cause
----------
0.30569320917129517, neglected children : and criminals
----------
0.3057822287082672, the street : with hate
----------
0.30584344267845154, an impact : and criminals
----------
ctn: 93
s_0
s_1
0.30597981810569763, a murder : and criminals
----------
0.30613574385643005, a baby : are people
----------
0.30626147985458374, using abortion : with hate
----------
0.3064349293708801, torturous disease : and criminals
----------
0.30664125084877014, the baby : and criminals
----------
0.3068138659000397, the pain : and criminals
----------
0.30694976449012756, since baby : and criminals
----------
ctn: 94
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
0.3070455491542816, a problem : a question
----------
0.30714255571365356, the apple : is murder
----------
0.3071717321872711, apple metaphor : coat hanger
----------
0.307232141494751, with crazymf : and criminals
----------
0.3073152005672455, his argument : and criminals
----------
0.30733442306518555, the one : a doctor
----------
0.3074013292789459, the farmer : the people
----------
0.30742576718330383, his apple : her mie
----------
0.3074392080307007, apple seed : coat hanger
----------
0.30752068758010864, a metaphor : the cause
----------
0.3076002299785614, for abortion : with hate
----------
0.30769458413124084, the orphanages : the people
----------
0.30776017904281616, improving orphanages : are people
----------
0.3078030049800873, making abortion : and lots
----------
0.30790477991104126, same time : this time
----------
0.3079965114593506, my point : the cause
----------
0.30808162689208984, up orphanages : with hate
----------
0.3081226944923401, pro-choice opponents : and criminals
----------
0.3082270622253418, that women : and criminals
----------
0.30831873416900635, their children : with hate
----------
0.308453768491745, for adoption : and criminals
----------
0.30856189131736755, an abortion : with hate
----------
0.30867624282836914, many children : and criminals
----------
0.30877095460891724, wrong color : the cause
----------
0.308849960565567, color age : are people
----------
0.308919221162796, age health : and criminals
----------
0.3090326488018036, health condition : are people
----------
0.3090861141681671, condition etc : and criminals
----------
0.309151291847229, when adoption : of abortions
----------
0.3092327117919922, an alternative : the cause
----------
0.30940428376197815, to abortion : with hate
----------
0.30965182185173035, the child : and criminals
----------
0.3098617494106293, the case : with hate
----------
0.3099339008331299, in orphanages : and criminals
----------
0.3099427819252014, or foster : are people
----------
0.3100231885910034, foster homes : and criminals
----------
0.3100806474685669, a lot : and lots
----------
0.3101706802845001, of mistreatment : and criminals
----------
0.3102247416973114, mistreatment occurs : and criminals
----------
0.3103850185871124, their child : of abortions
----------
0.31050628423690796, a life : are people
----------
0.31059640645980835, the one : and criminals
----------
0.31073471903800964, my conscience : with hate
----------
0.31086060404777527, an abortion : of abortions
----------
0.31095486879348755, that people : and criminals
----------
0.3110319972038269, take responsibility : and criminals
----------
0.31122252345085144, have children : and criminals
----------
0.31128382682800293, not reality : and criminals
----------
0.31141579151153564, a child : a question
----------
0.3114466667175293, make someone : make abortions
----------
0.31151679158210754, someone mature : are people
----------
0.3116556704044342, the child : are people
----------
0.3117561936378479, first place : and criminals
----------
0.3118504285812378, few people : and criminals
----------
0.3118942677974701, any sort : and criminals
----------
0.3120149075984955, of caliber : and criminals
----------
0.3121839165687561, the experience : and criminals
----------
0.31221696734428406, and love : and criminals
----------
0.3124050796031952, a child : and criminals
----------
0.3124796152114868, several people : and criminals
----------
0.3125077486038208, both men : are people
----------
0.3125484585762024, and women : and criminals
----------
0.31264764070510864, having children : and criminals
----------
0.3126930892467499, their actions : my gramps
----------
0.31275123357772827, every day : are people
----------
0.3127560615539551, their subtle : her mie
----------
0.3127644658088684, neglect lack : and lots
----------
0.31286412477493286, total commitment : and criminals
----------
0.3129952847957611, a parent : are people
----------
0.31306150555610657, parent etc : and criminals
----------
0.31308096647262573, for example : in austin
----------
0.31316980719566345, one man : are people
----------
0.31334546208381653, great father : and criminals
----------
0.3134616017341614, oldest son : are people
----------
0.3135397136211395, his wife : and criminals
----------
0.31368574500083923, second son : are people
----------
0.31389302015304565, anymore children : and criminals
----------
0.3139181137084961, made plans : of abortions
----------
0.31399017572402954, oldest son : their actions
----------
0.31402191519737244, his wife : my gramps
----------
0.3140796720981598, her mind : her self
----------
0.31412696838378906, this day : this girl
----------
0.3141660690307617, seven years : 4th week
----------
0.3142116367816925, youngest son : this girl
----------
0.3142945468425751, his son : are people
----------
0.3143036961555481, son name : take responsibility
----------
0.3143146336078644, this mean : this time
----------
0.31443122029304504, a person : are people
----------
0.3144831359386444, of caliber : and criminals
----------
0.31464651226997375, loving father : and criminals
----------
0.314719557762146, most part : and criminals
----------
0.31478574872016907, 2nd son : kill fetises
----------
0.31486815214157104, his fault : is murder
----------
0.31502386927604675, second child : are people
----------
0.315153568983078, conscious decision : and criminals
----------
0.31524351239204407, his part : and criminals
----------
0.31533095240592957, youngest son : are people
----------
0.31549376249313354, the child : are people
----------
0.31565067172050476, the parent : and criminals
----------
0.3158003091812134, parent resentment : and criminals
----------
ctn: 95
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
0.31593117117881775, the question : the cause
----------
0.3160308301448822, the killing : and lots
----------
0.3160977363586426, an animal : the people
----------
0.31613248586654663, or cluster : are people
----------
0.31623005867004395, of cells : and criminals
----------
0.31633737683296204, intelligent beings : and criminals
----------
0.31649214029312134, their life : and criminals
----------
0.31655704975128174, the end : the cause
----------
0.3166075646877289, a human : a question
----------
0.3167303800582886, a brain : a question
----------
0.316858172416687, the world : and criminals
----------
0.3169521689414978, individual sperm : is murder
----------
0.3170459270477295, solitary ovum : kill fetises
----------
0.3170686662197113, of course : of abortions
----------
0.31718316674232483, single cell : this time
----------
0.3171851336956024, is half : and mccormick
----------
0.3172762393951416, final product : the cause
----------
0.31740260124206543, fertilized egg : of abortions
----------
0.3174174427986145, a lone : a stance
----------
0.3174462616443634, lone cell : a stance
----------
0.3175445795059204, separate cells : proper protection
----------
0.31763607263565063, cancerous growth : of abortions
----------
0.3177179992198944, a human : a coat
----------
0.3177715539932251, small group : a story
----------
0.3178640604019165, of cells : and criminals
----------
0.3180292546749115, that person : and criminals
----------
0.318096786737442, no problem : a question
----------
0.3181985020637512, cancerous cells : and criminals
----------
0.3182157874107361, technically part : the cause
----------
0.3183795213699341, individual cells : and criminals
----------
0.318488746881485, individual cells : and criminals
----------
0.3185872733592987, an issue : and criminals
----------
0.3187187612056732, larger group : are people
----------
0.3188731074333191, of cells : and criminals
----------
0.31891998648643494, the human : the cause
----------
0.31904587149620056, a group : are people
----------
0.3191588819026947, of cells : and criminals
----------
0.3193742036819458, a child : and criminals
----------
0.3195286691188812, the group : are people
----------
0.3196273446083069, of cells : and criminals
----------
0.3197507858276367, a brain : the cause
----------
0.3198412358760834, the brain : my gramps
----------
0.31992465257644653, focal point : the cause
----------
0.31999024748802185, fair way : the cause
----------
ctn: 96
s_0
s_1
s_2
s_3
s_4
s_5
0.3200708329677582, an abortion : of abortions
----------
0.3202420771121979, other people : and criminals
----------
0.3203665316104889, the baby : the cause
----------
0.3204774856567383, a man : the people
----------
0.32054877281188965, pregnant wife : are people
----------
0.32056647539138794, murder- one : are people
----------
0.3207222521305084, the wife : and criminals
----------
0.3208759129047394, the baby : are people
----------
0.32098686695098877, a man : the people
----------
0.32106804847717285, his wife : a coat
----------
0.3211524784564972, into miscarriage : are people
----------
0.3211997151374817, physical abuse : and criminals
----------
0.32125717401504517, or drugs : and criminals
----------
0.3212767243385315, or something : and criminals
----------
0.3213503956794739, murder- murder : and criminals
----------
0.32151299715042114, the baby : are people
----------
0.321635901927948, the woman : the people
----------
0.3218066394329071, the baby : are people
----------
0.321810245513916, let face : what aspect
----------
0.3219154179096222, the government : and criminals
----------
0.3219696283340454, what okay : and criminals
----------
0.3221070170402527, that murder : and criminals
----------
ctn: 97
s_0
s_1
0.3222375810146332, a fetus : a question
----------
0.32237550616264343, same sense : are people
----------
0.3225032389163971, my kidney : of abortions
----------
0.32270169258117676, a person : and criminals
----------
0.3228193521499634, my kidney : an infection
----------
ctn: 98
s_0
s_1
0.3228389322757721, yr olds : are people
----------
0.3228766620159149, yr olds : and criminals
----------
0.3229152262210846, temper tantrums : and criminals
----------
0.3230031728744507, nazi germany : and criminals
----------
0.3230762481689453, previous rebuttal : the cause
----------
ctn: 99
s_0
0.32320865988731384, a government : and criminals
----------
0.3232969343662262, from ourselves : and criminals
----------
ctn: 100
s_0
s_1
s_2
0.3233146369457245, are thousands : are people
----------
0.3234175741672516, unwanted children : and criminals
----------
0.32353493571281433, about cats : and criminals
----------
0.323604553937912, and dogs : and criminals
----------
0.3237086832523346, your pets : and criminals
----------
0.32379981875419617, no one : and criminals
----------
0.32392701506614685, the puppies : and criminals
----------
0.32394224405288696, with people- : in humans
----------
0.3240962326526642, more babies : and criminals
----------
0.3242395520210266, than people : and criminals
----------
ctn: 101
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
s_29
s_30
s_31
s_32
s_33
s_34
s_35
0.32434046268463135, your question : with hate
----------
0.32441744208335876, a human : a question
----------
0.32448703050613403, a living : the cause
----------
0.3245106041431427, that breathes : and criminals
----------
0.3245265483856201, and feels : and criminals
----------
0.3245560824871063, those senses : the cause
----------
0.32456907629966736, senses people : and criminals
----------
0.324587881565094, have disabilities : make abortions
----------
0.3245927393436432, but feeling : and trefil
----------
0.3246995210647583, that people : and criminals
----------
0.32476767897605896, no feeling : and criminals
----------
0.324882835149765, not humans : and criminals
----------
0.32495957612991333, same thing : a story
----------
0.3251127004623413, the fetus : the cause
----------
0.3251168727874756, stats canada : cited ford
----------
0.32515400648117065, most cases : their actions
----------
0.32527971267700195, of abortions : and criminals
----------
0.3253605365753174, 12 weeks : are people
----------
0.3254545331001282, unborn baby : are people
----------
0.3255375027656555, a voice : a question
----------
0.32565686106681824, the thing : the people
----------
0.32568785548210144, what death : is murder
----------
0.32570701837539673, what sufferance : use abortion
----------
0.3257138133049011, i repeat : us government
----------
0.32584241032600403, your argument : with hate
----------
0.32587048411369324, were nothing : are people
----------
0.3260438144207001, the womb : the people
----------
0.32606014609336853, only something : are people
----------
0.3261266350746155, any memories : the cause
----------
0.3261738419532776, 12 months : the people
----------
0.32618284225463867, felt something : because someone
----------
0.32632797956466675, the fetus : the people
----------
0.32644203305244446, other argument : with hate
----------
0.32668352127075195, their guilt : and criminals
----------
0.32671692967414856, was decision : and criminals
----------
0.3268042504787445, the baby : the people
----------
0.3268173038959503, i gon : mie twenties
----------
0.3269096910953522, honest question : and criminals
----------
0.3270546793937683, a woman : are people
----------
0.3271080255508423, and depress : and criminals
----------
0.3271544575691223, short while : coat hanger
----------
0.3273274600505829, the child : are people
----------
0.32750454545021057, the mother : and criminals
----------
0.32753801345825195, mother suffering : are people
----------
0.3276492655277252, diverse reasons : and criminals
----------
0.32777169346809387, the doctor : the people
----------
0.32786399126052856, doctor situation : are people
----------
0.3280085623264313, any harm : and criminals
----------
0.32806968688964844, are humans : are people
----------
0.32809534668922424, in court : in mammals
----------
0.3282026946544647, the money : the people
----------
0.328368604183197, the patient : and criminals
----------
0.328434556722641, the thing : a coat
----------
0.32847869396209717, : operations : of abortions
----------
0.32850125432014465, have risks : the cause
----------
0.32860955595970154, the doctor : the people
----------
0.3287007808685303, good operation : are people
----------
0.3287111818790436, i gon : mie twenties
----------
0.32879161834716797, apologize beforehand : and criminals
----------
0.3288380801677704, said anything : and criminals
----------
0.32900750637054443, prematured babies : and criminals
----------
0.3292122781276703, a fetus : are people
----------
0.3292608857154846, know hiv : and criminals
----------
0.32940229773521423, healthy life : of abortions
----------
0.3294168710708618, i doubt : are people
----------
0.32954221963882446, 10000 cases : the cause
----------
0.329674631357193, those cases : are people
----------
0.3298337757587433, by childbirth : and criminals
----------
0.33002591133117676, the child : the people
----------
0.3301372826099396, the virus : an infection
----------
0.33031609654426575, the mother : are people
----------
0.3304099142551422, main cause : the cause
----------
0.33049896359443665, the transmission : the cause
----------
0.3306470811367035, the virus : the people
----------
ctn: 102
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
0.3307161331176758, those principals : and criminals
----------
0.33082377910614014, the pursuit : the people
----------
0.33103179931640625, of happiness : and criminals
----------
0.3311011791229248, the happiness : the cause
----------
0.3312916159629822, the mother : and criminals
----------
0.33139359951019287, a situation : the cause
----------
0.3315681517124176, the child : the people
----------
0.3317003846168518, bad life : of abortions
----------
0.3317921757698059, handle life : with hate
----------
0.33194488286972046, a child : with hate
----------
0.3320232033729553, the expenses : the people
----------
0.33214709162712097, a child : are people
----------
0.3322373628616333, young age : the people
----------
0.33238184452056885, american citizen : are people
----------
0.33253002166748047, in america : and criminals
----------
0.3326107859611511, living thing : the people
----------
0.3327938914299011, in america : and criminals
----------
0.3329852521419525, a citizen : and criminals
----------
0.33306241035461426, no rights : with hate
----------
0.33315733075141907, believe abortion : and criminals
----------
0.33321893215179443, my opinion : with hate
----------
0.3332666754722595, legally abortion : of abortions
----------
ctn: 103
s_0
0.3334030508995056, a fetus : are people
----------
0.3335206210613251, murder murder : and criminals
----------
0.33364465832710266, legal term : and criminals
----------
ctn: 104
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
0.3338252604007721, the mother : and criminals
----------
0.33402684330940247, legal person : and criminals
----------
0.3342415690422058, a fetus : and criminals
----------
0.33437907695770264, her rights : and criminals
----------
0.33460986614227295, potential child : and criminals
----------
0.334741473197937, a woman : are people
----------
0.3348454236984253, her efforts : and criminals
----------
0.33485934138298035, has nothing : doing things
----------
0.3350096642971039, with fate : and criminals
----------
0.33511340618133545, the mother : the people
----------
0.3352905213832855, the fetus : are people
----------
0.3354777991771698, the question : and criminals
----------
0.33568456768989563, the fetus : are people
----------
0.33580926060676575, has rights : and criminals
----------
0.33596816658973694, the mother : and criminals
----------
0.33606210350990295, it anything : are people
----------
0.33618682622909546, a woman : are people
----------
0.33625808358192444, traumatic rape : of abortions
----------
0.3363245129585266, her anger : and criminals
----------
0.33654776215553284, the fetus : and criminals
----------
0.33660459518432617, a stage : a question
----------
0.33671775460243225, her life : are people
----------
0.3368864059448242, a child : are people
----------
0.33699655532836914, the rape : of abortions
----------
0.3370375335216522, rape victim : are people
----------
0.33705535531044006, 13 year : this girl
----------
0.3371412456035614, old girl : are people
----------
0.33721959590911865, yes girl : are people
----------
0.33740130066871643, a mother : and criminals
----------
0.33751562237739563, the child : are people
----------
0.33757248520851135, an attack : and criminals
----------
0.3377065062522888, your life : are people
----------
0.3378196954727173, your mother : and criminals
----------
0.33802348375320435, the event : and criminals
----------
0.33814799785614014, the victim : are people
----------
0.33831024169921875, the child : and criminals
----------
0.3384428322315216, the attack : and criminals
----------
0.3386605381965637, the child : and criminals
----------
0.3388538360595703, the child : are people
----------
0.33895862102508545, child fault : and criminals
----------
0.3391294479370117, the mother : are people
----------
0.3392433524131775, mother fault : are people
----------
0.3393653631210327, entire life : are people
----------
0.3394196927547455, wrong place : the cause
----------
0.33949464559555054, wrong time : the people
----------
0.33956947922706604, a mistake : a question
----------
0.33967021107673645, another mistake : the cause
----------
0.3397941291332245, the question : of abortions
----------
0.3399093747138977, your premise : and criminals
----------
0.3400421440601349, identical conclusion : are people
----------
0.3401727080345154, a fetus : a question
----------
0.3403962552547455, a person : and criminals
----------
0.3405156135559082, therefore abortion : are people
----------
0.3406597971916199, not murder : and criminals
----------
0.3407942056655884, a zygote : a question
----------
0.3408818244934082, an amoeba : of people
----------
ctn: 105
s_0
0.3409300446510315, scientific consensus : with hate
----------
0.34101900458335876, when life : of abortions
----------
0.3410845398902893, begins abortion : are people
----------
ctn: 106
s_0
s_1
s_2
0.3411358892917633, the state : this time
----------
0.3411760926246643, of nature : are people
----------
0.34126517176628113, no rights : with hate
----------
0.3413848876953125, or obligations : and criminals
----------
0.3414328098297119, just things : are people
----------
0.3415396809577942, in society : are people
----------
0.34155842661857605, we grants : my gramps
----------
0.34165993332862854, grants rights : and criminals
----------
0.3417964279651642, our interests : and criminals
----------
0.341882586479187, brief lesson : a question
----------
0.3419173061847687, social contract : and criminals
----------
0.34202665090560913, contract theory : and criminals
----------
0.34210672974586487, grant rights : with hate
----------
0.34218135476112366, no sooner : and criminals
----------
0.3423008918762207, than viability : and criminals
----------
0.34248343110084534, the point : are people
----------
0.34261587262153625, the termination : the people
----------
0.34283608198165894, a pregnancy : are people
----------
0.34299227595329285, the termination : are people
----------
0.34308815002441406, the woman : are people
----------
0.34319108724594116, woman pregnant : are people
----------
ctn: 107
s_0
0.3433504104614258, the matter : with hate
----------
0.34341979026794434, if abortion : of abortions
----------
0.3435671925544739, a doctor : are people
----------
0.3436989188194275, the chance : and criminals
----------
0.34380918741226196, many women : and criminals
----------
0.34396812319755554, a baby : are people
----------
0.34404146671295166, riskier ways : with hate
----------
0.3442983627319336, the fetus : and criminals
----------
ctn: 108
s_0
0.34439507126808167, sexist alternatives : and criminals
----------
ctn: 109
s_0
s_1
0.34446343779563904, some scenarios : this time
----------
0.34463050961494446, unborn baby : and criminals
----------
0.3447774648666382, the stomach : the people
----------
0.3449673354625702, my mother : and criminals
----------
0.34503117203712463, used crack : with hate
----------
0.3451746702194214, a possibility : with hate
----------
0.34538769721984863, deformed child : and criminals
----------
0.3455682098865509, every baby : and criminals
----------
0.34571385383605957, safe environment : and criminals
----------
ctn: 110
s_0
s_1
s_2
0.34576037526130676, a living : a question
----------
0.3458591103553772, living organism : the people
----------
0.34595349431037903, a host : a question
----------
0.34609484672546387, host body : the cause
----------
0.3462326228618622, no one : are people
----------
0.34635505080223083, any problem : and criminals
----------
0.3463817834854126, getting rid : the cause
----------
0.3464876115322113, any leglislation : and criminals
----------
0.34659019112586975, governmental law : and criminals
----------
0.3467216491699219, or doctrine : and criminals
----------
0.3468312919139862, theological entity : and criminals
----------
0.34692543745040894, entity abortion : are people
----------
0.34708842635154724, a decision : and criminals
----------
0.34712401032447815, pregnant female : kill fetises
----------
0.3472268581390381, terminating pregnancy : are people
----------
0.34723302721977234, other ancient : the pre-embryo
----------
0.34730371832847595, ancient societies : the people
----------
ctn: 111
s_0
0.347351610660553, with jonjax71 : with hate
----------
ctn: 112
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
0.34748733043670654, the rape : the people
----------
0.3475647568702698, rape incest : of abortions
----------
0.3476228713989258, incest etc : and criminals
----------
0.3477378785610199, horrible things : and criminals
----------
0.3477843999862671, only considerations : a question
----------
0.34786325693130493, during abortions : with hate
----------
0.34801560640335083, the family : are people
----------
0.34816721081733704, the relationship : of abortions
----------
0.3482323884963989, social consequences : and criminals
----------
0.3482697308063507, economic consequences : the cause
----------
0.3483072817325592, the fact : the cause
----------
0.3484126031398773, that pregnancy : of abortions
----------
0.3484586477279663, medical consequences : and criminals
----------
0.3485536277294159, the time : the cause
----------
0.3486419916152954, whole life : an abortion
----------
0.34868329763412476, economic factors : the people
----------
0.34870752692222595, be tons : and lots
----------
0.34878289699554443, emotional damage : and criminals
----------
0.34893250465393066, the baby : the people
----------
0.3489757478237152, to term : and criminals
----------
0.34908658266067505, some people : and criminals
----------
0.34920698404312134, same standards : with hate
----------
0.3492605686187744, and sex : and criminals
----------
0.349377304315567, with condoms : and criminals
----------
0.3495459258556366, of marriage : with hate
----------
0.3495718240737915, as part : are people
----------
0.3497287631034851, healthy relationship : and criminals
----------
0.34981295466423035, personal values : the cause
----------
0.34995704889297485, and morals : and criminals
----------
0.3500259220600128, morals thing : and criminals
----------
0.35017672181129456, a woman : and criminals
----------
0.3502882719039917, an abortion : with hate
----------
0.35032474994659424, many years : are people
----------
0.3505176603794098, the decision : and criminals
----------
0.3505840301513672, many stories : with hate
----------
0.3506636321544647, many stories : and criminals
----------
0.35081225633621216, the woman : are people
----------
0.3509468734264374, the child : the people
----------
0.3510874807834625, whole life : and criminals
----------
0.3511437773704529, out resources : with hate
----------
0.35121482610702515, improving education : of abortions
----------
0.3513062596321106, supportive etc : with hate
----------
0.35135790705680847, the things : the people
----------
0.35138383507728577, more needs : of abortions
----------
0.3514869213104248, these women : and criminals
----------
0.3516246974468231, unwanted pregnancies : and criminals
----------
0.3516697883605957, the way : the cause
----------
0.3518632650375366, a child : are people
----------
0.3520210087299347, a baby : are people
----------
0.3521089553833008, pro choice : are people
----------
0.3521987795829773, convincing people : and criminals
----------
0.3523809313774109, kill children : and criminals
----------
0.3524066209793091, giving education : make abortions
----------
0.3525054156780243, preventing pregnancies : and criminals
----------
0.35257208347320557, by use : are people
----------
0.35268065333366394, of education : and criminals
----------
0.35286229848861694, on contraceptives : and criminals
----------
0.35301482677459717, and abstinence : and criminals
----------
0.353087455034256, an abstinence : with hate
----------
0.3531496226787567, only education : are people
----------
0.35325729846954346, have merits : and criminals
----------
0.3533371686935425, as people : and criminals
----------
0.3535099923610687, up plan : and criminals
----------
0.3536117970943451, like condoms : and criminals
----------
0.3536997437477112, pro choice : are people
----------
0.35382726788520813, prevent pregnancies : and criminals
----------
0.35395294427871704, before abortion : and criminals
----------
0.35402098298072815, an *option* : and criminals
----------
0.3541300296783447, informed option : the cause
----------
0.3542327284812927, with support : the cause
----------
0.35435476899147034, either choice : are people
----------
0.35447990894317627, the person : and criminals
----------
0.3546360433101654, only option : and criminals
----------
ctn: 113
s_0
s_1
s_2
s_3
s_4
s_5
s_6
0.3548256456851959, a child : are people
----------
0.3549973666667938, the womb : are people
----------
0.355097234249115, the sperm : the cause
----------
0.35515710711479187, and egg : and lots
----------
0.35520708560943604, egg beforehand : are people
----------
0.35530519485473633, the point : the cause
----------
0.35535016655921936, harmless killing : of abortions
----------
0.35548457503318787, of cells : and criminals
----------
0.35549622774124146, and becomes : the people
----------
0.3556012511253357, becomes murder : and criminals
----------
0.35569027066230774, the time : the people
----------
0.35579076409339905, of birth : and criminals
----------
0.3559001088142395, the mother : the people
----------
0.3560909628868103, the child : are people
----------
0.35618188977241516, reasonable reason : and lots
----------
0.3563873767852783, her choice : and criminals
----------
0.35646766424179077, my views : with hate
----------
0.3566476106643677, the father : and criminals
----------
0.35679543018341064, father position : and criminals
----------
0.35693028569221497, the dabate : are people
----------
0.35708335041999817, the mother : are people
----------
0.3572688698768616, the chance : with hate
----------
0.3573726415634155, the lump : the people
----------
0.35746848583221436, of life : and criminals
----------
0.3575485348701477, use words : and criminals
----------
0.35770857334136963, the feelings : and criminals
----------
0.3578718900680542, in people : and criminals
----------
0.357887327671051, anger towards : and criminals
----------
0.3579639792442322, towards pro-choicers : and criminals
----------
0.3580029606819153, pro-choicers shock : and criminals
----------
0.35811036825180054, the idea : and criminals
----------
0.3583354949951172, a child : and criminals
----------
0.3585222661495209, proper argument : and criminals
----------
0.35864344239234924, sway people : and criminals
----------
0.3588014841079712, your side : and criminals
----------
0.358883261680603, the use : with hate
----------
0.3589993715286255, of emotion : and criminals
----------
0.3591286540031433, not rationalism : and criminals
----------
ctn: 114
s_0
0.3591940104961395, the portion : the people
----------
0.3592812716960907, the constitution : the people
----------
0.3594212234020233, prenatal development : and criminals
----------
ctn: 115
s_0
0.35948532819747925, if life : of abortions
----------
0.35953769087791443, at conception : of abortions
----------
0.35970017313957214, that life : and criminals
----------
0.3598831295967102, a mother : and criminals
----------
0.36005231738090515, a miscarriage : of abortions
----------
0.3601156175136566, negligent homicide : and criminals
----------
0.36019983887672424, involuntary manslaughter : and criminals
----------
ctn: 116
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
0.36025315523147583, killing something : with hate
----------
0.3604850172996521, a person : and criminals
----------
0.3605077564716339, of course : of abortions
----------
0.360668420791626, your logic : the cause
----------
0.360757976770401, a month : a coat
----------
0.3607756495475769, month passes : weeks gestation
----------
0.3609330654144287, a woman : are people
----------
0.3610879182815552, innocent egg : and criminals
----------
0.3611829876899719, egg cell : are people
----------
0.3613004684448242, cell future : are people
----------
0.36143913865089417, the holocaust : with hate
----------
0.36158570647239685, male masturbation : and criminals
----------
0.3616875410079956, potential baby : the government
----------
0.3618660569190979, a baby : are people
----------
0.36205539107322693, a corpse : are people
----------
0.36222195625305176, the woman : and criminals
----------
0.362308531999588, woman body : are people
----------
0.3624463677406311, that pregnancy : and criminals
----------
0.362516313791275, to term : with hate
----------
0.3627232611179352, her autonomy : with hate
----------
0.36276188492774963, the smarter : the government
----------
0.3627648651599884, and humane : compare humans
----------
0.36287879943847656, humane solution : are people
----------
0.36292511224746704, comprehensive sex : with hate
----------
0.36302825808525085, free contraception : with hate
----------
0.3631134033203125, the amount : are people
----------
0.36323490738868713, unwanted pregnancies : and criminals
----------
0.36336031556129456, first place : and criminals
----------
0.3634325861930847, an abortion : of abortions
----------
ctn: 117
s_0
s_1
0.36346369981765747, united states : the government
----------
0.36353427171707153, america pro-choice : with hate
----------
0.3636724650859833, constitutional way : and criminals
----------
0.36374631524086, the world : this time
----------
0.3638007938861847, world people : are people
----------
ctn: 118
s_0
s_1
s_2
s_3
0.36391064524650574, other choices : the cause
----------
0.3640230894088745, are adoption : are people
----------
0.3641502559185028, many children : and criminals
----------
0.36428460478782654, an abortion : with hate
----------
0.3643636405467987, the chance : the cause
----------
0.3644038736820221, i guess : are people
----------
0.3644900619983673, one womans : kill fetises
----------
0.36462196707725525, womans pregnancy : an infection
----------
ctn: 119
s_0
s_1
s_2
s_3
s_4
s_5
0.3647305965423584, the constitution : the cause
----------
0.36479318141937256, the declaration : the cause
----------
0.36485761404037476, of independance : of abortions
----------
0.36495885252952576, the constitution : the cause
----------
0.36501583456993103, the powers : the cause
----------
0.3650880753993988, the government : the cause
----------
0.36517298221588135, the government : the people
----------
0.3652571439743042, not-outlawing abortion : of abortions
----------
0.3653923273086548, a non-action : with hate
----------
0.3654542863368988, by government : and criminals
----------
0.3655555248260498, the constitution : the cause
----------
0.36560744047164917, govern states : are people
----------
0.36566969752311707, states cities : are people
----------
0.36574143171310425, or individuals : and criminals
----------
ctn: 120
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
0.365884929895401, a bigot : and criminals
----------
0.36596232652664185, that people : and criminals
----------
0.3661021292209625, your actions : and criminals
----------
0.36615657806396484, right thing : and criminals
----------
0.3661586046218872, bible mentions : that answers
----------
0.36618107557296753, mentions nothing : and lots
----------
0.36630702018737793, about tolerance : with hate
----------
0.3664296269416809, the bible : the people
----------
0.36655324697494507, a lot : are people
----------
0.3666575849056244, about love : and criminals
----------
0.36670997738838196, specific parable : proper protection
----------
0.3667478561401367, good samaritan : illigal try
----------
0.3668549656867981, undesirable person : and criminals
----------
0.3669050931930542, a kind : are people
----------
0.3670414984226227, good neighbour : and criminals
----------
0.3670760989189148, second greatest : the cause
----------
0.3672035038471222, greatest commandment : are people
----------
0.36736637353897095, your neighbour : and criminals
----------
0.36741402745246887, offering support : selves abortions
----------
0.36743250489234924, and help : proper protection
----------
0.36750152707099915, young women : are people
----------
0.36765316128730774, in life : with hate
----------
0.3678169846534729, that support : of abortions
----------
0.3679276406764984, an abortion : and criminals
----------
0.3679760992527008, if people : and criminals
----------
0.36800074577331543, god laws : doing things
----------
0.36807653307914734, good choices : are people
----------
0.36820778250694275, no need : and criminals
----------
0.3682348430156708, new testament : proper protection
----------
0.36828428506851196, testament laws : selves abortions
----------
0.36845481395721436, your problem : the cause
----------
0.3685067296028137, not everyone : and criminals
----------
0.36860647797584534, these laws : and criminals
----------
0.3686603009700775, all christians : and criminals
----------
0.3687976598739624, these laws : and criminals
----------
0.3689000606536865, that people : and criminals
----------
0.3689585030078888, just christians : and criminals
----------
0.36896464228630066, should petition : to privacy
----------
0.3690980076789856, for education : and criminals
----------
0.36910587549209595, education concerning : and lots
----------
0.3692038655281067, just pregnancy : are people
----------
0.36924630403518677, what sort : are people
----------
0.36939531564712524, of cost : and criminals
----------
0.3696097731590271, a child : and criminals
----------
0.3697453439235687, the risks : with hate
----------
0.36982235312461853, of stds : and criminals
----------
0.36997589468955994, the use : are people
----------
0.37011775374412537, of contraceptives : and criminals
----------
0.37020060420036316, the heart : the cause
----------
0.37024375796318054, and people : and criminals
----------
0.3703269362449646, young girl : are people
----------
0.37037816643714905, poor education : an infection
----------
0.37046489119529724, older man : a coat
----------
0.3705155551433563, his love : and criminals
----------
0.37058448791503906, her naivety : with hate
----------
0.3706229329109192, had sex : with hate
----------
0.3706870973110199, no way : are people
----------
0.3707643747329712, take care : with hate
----------
0.37089666724205017, the baby : are people
----------
0.37091803550720215, or deal : are people
----------
0.3709593415260315, social implications : and criminals
----------
0.3710583746433258, an abortion : with hate
----------
0.3710750937461853, her evil : my gramps
----------
0.37110304832458496, her wicked : my gramps
----------
0.371198832988739, your intention : a story
----------
0.37123891711235046, many times : the people
----------
0.3713073432445526, the things : the people
----------
0.37146756052970886, a woman : and criminals
----------
0.3715609610080719, an abortion : of abortions
----------
0.37157344818115234, -- lets : and criminals
----------
0.37166327238082886, the example : are people
----------
0.3717386722564697, of someone : and criminals
----------
0.37184035778045654, a church : a question
----------
0.3718840479850769, church pastor : are people
----------
0.37197503447532654, the age : are people
----------
0.372058242559433, of consent : are people
----------
0.3721250593662262, his love : and criminals
----------
0.37215569615364075, had sex : with hate
----------
0.37219247221946716, here repercussions : the cause
----------
0.3723186254501343, the baby : the cause
----------
0.3724285662174225, social outcast : and criminals
----------
0.37263625860214233, the church : and criminals
----------
0.37273988127708435, take care : with hate
----------
0.37300071120262146, the child : and criminals
----------
0.3731989562511444, her age : and criminals
----------
0.3732384443283081, stop school : are people
----------
0.37343090772628784, the child : and criminals
----------
0.3736139237880707, the man : and criminals
----------
0.3737979829311371, his power : and criminals
----------
0.37397730350494385, a relationship : and criminals
----------
0.3740231394767761, fight evil : and criminals
----------
0.37405791878700256, more evil : and criminals
----------
0.37411051988601685, an example : are people
----------
0.37415724992752075, god commands : and criminals
----------
0.37425923347473145, hateful voice : and criminals
----------
0.3744161128997803, a crowd : and criminals
----------
0.3745514452457428, young women : and criminals
----------
ctn: 121
s_0
s_1
s_2
0.37464743852615356, my fault : is murder
----------
0.37475815415382385, your argument : with hate
----------
0.37482261657714844, beyond salvation : and criminals
----------
0.37487074732780457, useless arguments : of abortions
----------
0.3749910891056061, simplest solution : are people
----------
ctn: 122
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
0.37507668137550354, in abortion : of abortions
----------
0.375103235244751, abortion type : are people
----------
0.37523001432418823, type issues : and criminals
----------
0.3753288984298706, the farmer : the people
----------
0.37542036175727844, own seed : and criminals
----------
0.3754606544971466, outside judgement : proper protection
----------
0.3755739629268646, no one : and criminals
----------
0.37567654252052307, is pro-abortion : and criminals
----------
0.37574565410614014, no one : are people
----------
0.37580880522727966, see life : of abortions
----------
0.3758906126022339, whether people : and criminals
----------
0.37604445219039917, that choice : and criminals
----------
0.37615084648132324, with abortion : with hate
----------
0.3762787878513336, this right : and criminals
----------
0.37644803524017334, for women : and criminals
----------
0.3765895366668701, and families : and criminals
----------
0.37682461738586426, ilegal abortions : and criminals
----------
0.37686532735824585, black market : and criminals
----------
0.37698259949684143, market doctors : and criminals
----------
0.3770272731781006, some women : the people
----------
0.3771991729736328, the fetus : are people
----------
0.37724724411964417, with clothes : with hate
----------
0.37728333473205566, clothes hangers : an infection
----------
0.3774087727069855, a threat : with hate
----------
0.3775923550128937, the child : and criminals
----------
0.377807080745697, the mother : and criminals
----------
0.3778429329395294, these women : her self
----------
0.37793609499931335, safe environment : the cause
----------
0.3781028687953949, the decision : the people
----------
0.3782796859741211, a child : are people
----------
0.3783743381500244, a child : a coat
----------
0.3784969747066498, for adoption : and criminals
----------
0.3785422146320343, some mothers : the people
----------
0.3786652088165283, a child : are people
----------
0.378693163394928, financial reasons : and criminals
----------
0.3787026107311249, reasons others : some scientists
----------
0.3787851929664612, behavorial age : and criminals
----------
0.37883466482162476, and drug : and criminals
----------
0.3789048194885254, drug problems : and criminals
----------
0.3789931535720825, these people : and criminals
----------
ctn: 123
s_0
s_1
s_2
0.3790428042411804, a womans : a question
----------
0.3791118264198303, reproductive rights : of abortions
----------
0.3792245388031006, motivated governments : and criminals
----------
0.37935787439346313, the globe : and criminals
----------
0.3794134855270386, such things : and criminals
----------
0.37947091460227966, make women : with hate
----------
0.379543274641037, wear berka : with hate
----------
0.3795579671859741, if anything : because someone
----------
0.37960153818130493, anything abortion : of abortions
----------
0.3796517550945282, litmus test : the cause
----------
0.3797290623188019, if religion : of abortions
----------
0.37979599833488464, united states : are people
----------
0.3798733353614807, reproductive rights : and criminals
----------
0.3800361156463623, of women : and criminals
----------
0.3800637722015381, its fair : with hate
----------
0.38025104999542236, promoting mission : and criminals
----------
0.3803388774394989, in countries : and criminals
----------
0.3804834187030792, motivated politics : and criminals
----------
0.38059768080711365, promoting actions : and criminals
----------
0.3806646168231964, an individuals : and criminals
----------
0.38074934482574463, the right : with hate
----------
0.3808208405971527, have access : and criminals
----------
0.38101309537887573, to abortions : and criminals
----------
ctn: 124
s_0
s_1
0.3810831606388092, unwanted child : and criminals
----------
0.3811168968677521, to term : of abortions
----------
0.38115131855010986, an effect : with hate
----------
0.3813256025314331, other people : and criminals
----------
0.381409615278244, the world : the people
----------
0.3815295398235321, illegal abortion : and criminals
----------
ctn: 125
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
0.381621778011322, your grammar : the cause
----------
0.38175126910209656, your point : the cause
----------
0.38185861706733704, the doctors : and criminals
----------
0.3819844424724579, that abortion : and criminals
----------
0.382130891084671, best option : with hate
----------
0.382163405418396, fifteen year : this time
----------
0.3822880983352661, some cases : and criminals
----------
0.38233327865600586, the time : the people
----------
0.3824978470802307, the child : are people
----------
0.38259559869766235, get abortions : with hate
----------
0.3826550245285034, sketchy back-alleys : of abortions
----------
0.3828114867210388, for abortions : and criminals
----------
0.38288676738739014, when abortion : of abortions
----------
0.3829454779624939, what women : and criminals
----------
0.3830346465110779, many women : are people
----------
0.38308194279670715, this day : this girl
----------
0.3831329047679901, day women : are people
----------
0.3832009434700012, a year : the cause
----------
0.3833225965499878, illegal abortions : and criminals
----------
0.3834531605243683, a fetus : a question
----------
0.3834981322288513, any type : and lots
----------
0.3835720419883728, 8 months : the cause
----------
0.3836783468723297, in utero : are people
----------
0.38372567296028137, low-level thinking : the cause
----------
0.3837703466415405, some women : the people
----------
0.38385823369026184, an abortion : with hate
----------
0.38404932618141174, their choice : and criminals
----------
0.38414016366004944, an abortion : of abortions
----------
0.38432577252388, a child : and criminals
----------
0.3844466507434845, in hell : and criminals
----------
0.3846413493156433, 18 years : and criminals
----------
0.38473382592201233, these days : the cause
----------
0.38479411602020264, an option : the cause
----------
0.3848504424095154, option foster : are people
----------
0.3849920332431793, foster homes : and criminals
----------
0.385098934173584, the brim : with hate
----------
0.3852351903915405, unwanted kids : and criminals
----------
0.3853471279144287, like juno : are people
----------
0.38547709584236145, sensible options : and criminals
----------
0.385642409324646, unwanted pregnancy : and criminals
----------
ctn: 126
s_0
s_1
0.38571107387542725, legalizing drugs : with hate
----------
0.3858240246772766, saving lives : with hate
----------
0.38585880398750305, keeping abortion : of abortions
----------
0.38593876361846924, save lives : with hate
----------
0.38604798913002014, the need : with hate
----------
0.3862415552139282, a child : are people
----------
0.38638028502464294, same time : are people
----------
ctn: 127
s_0
s_1
s_2
s_3
0.3864932358264923, their body : the cause
----------
0.3865094780921936, not yours : is murder
----------
ctn: 128
s_0
ctn: 129
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
0.3866208791732788, the thing : the cause
----------
0.3867402970790863, second person : and criminals
----------
0.3868573307991028, the baby- : the people
----------
0.3870525062084198, the body : are people
----------
0.387279748916626, the woman : and criminals
----------
0.38743773102760315, the woman : are people
----------
0.38748303055763245, deciding everything : is murder
----------
0.3876765966415405, the baby : the people
----------
0.38780418038368225, the baby : the people
----------
0.38782960176467896, in her- : of abortions
----------
0.3879862427711487, her body : are people
----------
0.388102263212204, her body : an infection
----------
0.38825029134750366, her body : the cause
----------
0.38839027285575867, her body : an infection
----------
0.38842567801475525, give blood : of abortions
----------
0.3885020315647125, or organs : and criminals
----------
0.3885488212108612, makes decisions : with hate
----------
0.38870811462402344, the baby : the people
----------
0.388843834400177, her choice : with hate
----------
0.3889714181423187, the baby : the people
----------
0.3890531659126282, an extension : the cause
----------
0.3891499936580658, her body : an infection
----------
0.38921406865119934, her rules : the cause
----------
0.38927826285362244, the decisions : a question
----------
0.3894244432449341, the baby : of abortions
----------
ctn: 130
s_0
s_1
s_2
0.3894411027431488, i guess : a story
----------
0.38960036635398865, weird argument : with hate
----------
ctn: 0
s_0
s_1
s_2
0.25597625970840454, the fetus : the anti-choicers
----------
0.22168926894664764, a part : when someone
----------
ctn: 1
s_0
s_1
s_2
ctn: 2
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
ctn: 3
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
ctn: 4
s_0
ctn: 5
s_0
s_1
s_2
s_3
s_4
ctn: 6
s_0
ctn: 7
s_0
s_1
s_2
ctn: 8
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 9
s_0
ctn: 10
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
ctn: 11
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 12
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
ctn: 13
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 14
s_0
ctn: 15
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
s_29
s_30
s_31
s_32
ctn: 16
s_0
s_1
s_2
s_3
s_4
s_5
s_6
ctn: 17
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 18
s_0
s_1
s_2
s_3
ctn: 19
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 20
s_0
s_1
ctn: 21
s_0
ctn: 22
s_0
s_1
s_2
ctn: 23
s_0
s_1
s_2
s_3
s_4
ctn: 24
s_0
ctn: 25
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
ctn: 26
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
ctn: 27
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
ctn: 28
s_0
s_1
ctn: 29
s_0
s_1
s_2
s_3
s_4
s_5
s_6
ctn: 30
s_0
s_1
s_2
s_3
ctn: 31
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
ctn: 32
s_0
s_1
ctn: 33
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
ctn: 34
s_0
s_1
s_2
s_3
ctn: 35
s_0
s_1
s_2
s_3
s_4
s_5
s_6
ctn: 36
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
ctn: 37
s_0
s_1
s_2
ctn: 38
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
ctn: 39
s_0
s_1
ctn: 40
s_0
s_1
ctn: 41
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 42
s_0
ctn: 43
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
ctn: 44
s_0
ctn: 45
s_0
s_1
ctn: 46
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
ctn: 47
s_0
s_1
ctn: 48
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 49
s_0
s_1
ctn: 50
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
ctn: 51
s_0
s_1
s_2
s_3
s_4
ctn: 52
s_0
s_1
s_2
s_3
s_4
s_5
ctn: 53
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
ctn: 54
s_0
s_1
0.20011940598487854, that abortion : and criminals
----------
0.2003178745508194, your rebuttal : and criminals
----------
0.2004435658454895, wrote words : and criminals
----------
ctn: 55
s_0
0.20049859583377838, it alredy : illigal try
----------
ctn: 56
s_0
0.20066411793231964, yes zombieslayer : and criminals
----------
0.20083953440189362, womans choise : and criminals
----------
0.20102784037590027, be consequences : and criminals
----------
0.20123310387134552, her choice : and criminals
----------
ctn: 57
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.20142818987369537, what life : and criminals
----------
0.20154619216918945, god descision : and criminals
----------
0.20167255401611328, biologically science : and criminals
----------
0.20187443494796753, the fetus : and criminals
----------
0.20206354558467865, seperate life : are people
----------
0.20225276052951813, its mother : and criminals
----------
0.20246626436710358, the fetus : are people
----------
0.20254333317279816, own seperate : and criminals
----------
0.20270505547523499, seperate body : and criminals
----------
0.2028946578502655, are babies : and criminals
----------
0.20303864777088165, three weeks : the people
----------
0.2032291293144226, the pregnancy : are people
----------
0.20345495641231537, the fetus : and criminals
----------
0.2036147564649582, developed eyes : and criminals
----------
0.20379172265529633, and appendages : and criminals
----------
0.203956738114357, actual humans : and criminals
----------
0.2040831297636032, what difference : and criminals
----------
0.20430082082748413, the child : and criminals
----------
0.20447339117527008, the womb : and criminals
----------
0.20464126765727997, a life : and criminals
----------
0.2048354595899582, that murder : and criminals
----------
ctn: 58
s_0
0.20499639213085175, up burden : and criminals
----------
0.20518319308757782, of proof : and criminals
----------
ctn: 59
s_0
s_1
s_2
0.2053677886724472, an abortion : and criminals
----------
0.20556075870990753, abortion debate : and criminals
----------
0.20576511323451996, two weeks : and criminals
----------
0.20595376193523407, same people : and criminals
----------
0.20610763132572174, same arguments : and criminals
----------
0.20626415312290192, and rebuttals : and criminals
----------
0.2064206302165985, last debate : and criminals
----------
0.20661038160324097, moral issue : and criminals
----------
0.2068043202161789, political issue : and criminals
----------
0.2069617509841919, convincing anyone : and criminals
----------
0.20717747509479523, their morals : and criminals
----------
0.2073543518781662, this debate : and criminals
----------
0.20752255618572235, debate pointless : and criminals
----------
ctn: 60
s_0
0.2076442986726761, anti-death penalty : and criminals
----------
0.20779430866241455, penalty pro-choice : and criminals
----------
ctn: 61
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
0.20796988904476166, logical explanation : and criminals
----------
0.20811614394187927, why abortion : and criminals
----------
0.2082846462726593, that murder : and criminals
----------
0.20838893949985504, ) murder : and criminals
----------
0.20855888724327087, unjustified killing : and criminals
----------
0.2087354212999344, another human : and criminals
----------
0.20893366634845734, living organism : and criminals
----------
0.20909099280834198, the dna : and criminals
----------
0.20931096374988556, the species : and criminals
----------
0.20943279564380646, species homo : and criminals
----------
0.209574356675148, homo sapiens : and criminals
----------
0.2097703069448471, an organism : and criminals
----------
0.20996280014514923, another organism : and criminals
----------
0.21015802025794983, an organism : and criminals
----------
0.21030516922473907, cellular level : are people
----------
0.21046042442321777, to changes : and criminals
----------
0.21063043177127838, its surroundings : and criminals
----------
0.21078188717365265, use energy : with hate
----------
0.21094386279582977, its surroundings : and criminals
----------
0.21109288930892944, at conception : of abortions
----------
0.2112683802843094, fertilized egg : and criminals
----------
0.2113940417766571, human dna : and criminals
----------
0.21158549189567566, a life : and criminals
----------
0.21170508861541748, life distinct : are people
----------
0.2118988335132599, its mother : and criminals
----------
0.21209733188152313, unborn child : and criminals
----------
0.21227027475833893, the mother : and criminals
----------
0.21240052580833435, and vice : and criminals
----------
0.2125425487756729, vice versa : are people
----------
0.21275010704994202, new cells : and criminals
----------
0.21291932463645935, its surroundings : and criminals
----------
0.2130780667066574, cellular level : and criminals
----------
0.21321836113929749, using energy : with hate
----------
0.21338967978954315, its surroundings : and criminals
----------
0.21349211037158966, at conception : and criminals
----------
0.21367283165454865, fertilized egg : and criminals
----------
0.21387630701065063, unborn child : and criminals
----------
0.21408255398273468, a crime : and criminals
----------
0.21421833336353302, ) abortion : and criminals
----------
0.21443746984004974, the child : and criminals
----------
0.21457648277282715, ) abortion : and criminals
----------
0.21475301682949066, unjustified killing : and criminals
----------
0.21496015787124634, human child : and criminals
----------
0.21514743566513062, another human : and criminals
----------
0.21526843309402466, thus abortion : with hate
----------
0.21545711159706116, is murder : and criminals
----------
0.21557720005512238, if murder : and criminals
----------
0.21574725210666656, then abortion : and criminals
----------
ctn: 62
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.21588720381259918, haha funny : and criminals
----------
0.21603037416934967, funny sorry : and criminals
----------
0.21613115072250366, only thing : are people
----------
0.21627691388130188, every mistake : and criminals
----------
0.21637490391731262, the past : the people
----------
0.21650776267051697, only thing : are people
----------
0.21669864654541016, the future : and criminals
----------
0.21685540676116943, better chocies : with hate
----------
0.21701374650001526, same mistake : and criminals
----------
0.21711768209934235, mistake twice : are people
----------
0.2173154056072235, a baby : and criminals
----------
0.2173677384853363, just becasue : are people
----------
0.21751265227794647, becasue something : and criminals
----------
0.21772706508636475, many people : and criminals
----------
0.21791358292102814, right thing : and criminals
----------
0.21812613308429718, a animal : and criminals
----------
0.21832792460918427, human baby : and criminals
----------
0.21850694715976715, those people : and criminals
----------
0.21869675815105438, a change : are people
----------
0.2188369184732437, same thing : and criminals
----------
ctn: 63
s_0
s_1
s_2
s_3
0.2189525067806244, wrong becasue : are people
----------
0.21911689639091492, the chance : and criminals
----------
0.21931549906730652, my point : and criminals
----------
0.219513401389122, of view : and criminals
----------
ctn: 64
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
0.21965761482715607, your relatives : and criminals
----------
0.21979151666164398, fifteen year : are people
----------
0.2199728935956955, the right : and criminals
----------
0.2202235460281372, the fetus : and criminals
----------
0.22034375369548798, a horrific : and criminals
----------
0.22052106261253357, horrific experience : and criminals
----------
0.22072023153305054, the pain : and criminals
----------
0.22085800766944885, and humiliaty : and criminals
----------
0.22097130119800568, of course : and criminals
----------
0.22116051614284515, same experience : and criminals
----------
0.2213008850812912, nine months : are people
----------
0.22143596410751343, of hell : and criminals
----------
0.22168666124343872, the fetus : and criminals
----------
0.2218404859304428, giving birth : with hate
----------
0.22204096615314484, the baby : and criminals
----------
0.22223468124866486, its face : and criminals
----------
0.22238995134830475, the night : and criminals
----------
0.22256828844547272, the child : and criminals
----------
0.22275306284427643, two individuals : and criminals
----------
0.22293297946453094, a man : and criminals
----------
0.2230483591556549, took advantage : and criminals
----------
0.22323642671108246, to death : and criminals
----------
0.22333677113056183, bed time : with hate
----------
0.2235201895236969, time story : and criminals
----------
0.22357922792434692, i bet : and criminals
----------
0.22372938692569733, eat eggs : and criminals
----------
0.22394289076328278, the fetus : are people
----------
0.22410117089748383, a thing : and criminals
----------
0.22429470717906952, that abortion : and criminals
----------
0.22448238730430603, about rape : and criminals
----------
0.22462977468967438, rape victims : and criminals
----------
0.22475536167621613, the memories : and criminals
----------
0.22487331926822662, with people : and criminals
----------
0.22501856088638306, them murderers : and criminals
----------
0.22518901526927948, their fault : and criminals
----------
0.2253289818763733, such acts : and criminals
----------
ctn: 65
s_0
0.22551681101322174, god-given laws : and criminals
----------
0.22570927441120148, the constitution : and criminals
----------
ctn: 66
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
0.22586394846439362, this question : and criminals
----------
0.2260684072971344, the fetus : are people
----------
0.2262786477804184, the abortion : and criminals
----------
0.22643977403640747, takes place : and criminals
----------
0.22662042081356049, further details : and criminals
----------
0.2267039269208908, a step : the cause
----------
0.22691208124160767, controversial issue : and criminals
----------
0.22711855173110962, define life : and criminals
----------
0.22731849551200867, human adult : and criminals
----------
0.22745424509048462, an adult : and criminals
----------
0.2276277393102646, adult coma : and criminals
----------
0.22777880728244781, coma patient : and criminals
----------
0.22793471813201904, an example : with hate
----------
0.22813329100608826, a heartbeat : are people
----------
0.22827260196208954, heartbeat regardless : are people
----------
0.22846278548240662, the assistance : and criminals
----------
0.22862780094146729, of machinery : and criminals
----------
0.22876878082752228, the mayo : of abortions
----------
0.22892557084560394, the heart : are people
----------
0.22910656034946442, five weeks : with hate
----------
0.22925958037376404, the mayo : the people
----------
0.2293207347393036, clinic numbers : are people
----------
0.22943593561649323, the weeks : the people
----------
0.22960856556892395, the start : are people
----------
0.2297656536102295, last period : are people
----------
0.2299438714981079, that conception : and criminals
----------
0.230066180229187, in week : with hate
----------
0.23027369379997253, the heartbeat : and criminals
----------
0.2304397076368332, 4th week : are people
----------
0.23062068223953247, after conception : with hate
----------
0.23081214725971222, this date : are people
----------
0.23093350231647491, the range : are people
----------
0.23108623921871185, first trimester : with hate
----------
0.2312423139810562, a time : are people
----------
0.23143914341926575, time frame : and criminals
----------
0.23164205253124237, an abortion : and criminals
----------
0.2318190485239029, take place : and criminals
----------
0.23193234205245972, of course : and criminals
----------
0.2320466935634613, course brain : and criminals
----------
0.23221030831336975, brain activity : and criminals
----------
0.23241455852985382, the fetus : the people
----------
0.23259592056274414, twenty-five weeks : are people
----------
0.23280411958694458, after conception : and criminals
----------
0.23289965093135834, considers brainwaves : and criminals
----------
0.23300719261169434, the defining : the cause
----------
0.23322579264640808, of life : and criminals
----------
0.23342183232307434, the fetus : and criminals
----------
0.233559250831604, this defense : and criminals
----------
0.23368367552757263, consider coma : and criminals
----------
0.23382049798965454, coma patients : and criminals
----------
0.23395408689975739, no brain : and criminals
----------
0.23409423232078552, brain activity : and criminals
----------
0.23416998982429504, is evidence : with hate
----------
0.23435859382152557, their lives : and criminals
----------
0.23452265560626984, two scenarios : and criminals
----------
0.2346923053264618, a stance : with hate
----------
0.2348906397819519, corresponding stance : and criminals
----------
0.23508286476135254, the conditions : and criminals
----------
0.23529566824436188, of life : and criminals
----------
0.23545555770397186, this point : with hate
----------
0.23562484979629517, basic assumption : and criminals
----------
0.23577848076820374, the beating : the people
----------
0.23596197366714478, the heart : and criminals
----------
0.23617148399353027, a life : and criminals
----------
0.236348494887352, the levels : and criminals
----------
0.23645088076591492, of brain : and criminals
----------
0.23656144738197327, brain activity : and criminals
----------
0.23674382269382477, decisive measure : and criminals
----------
0.2368900328874588, the arguments : and criminals
----------
0.23707307875156403, pro-choice side : and criminals
----------
0.23724862933158875, the woman : and criminals
----------
0.23741847276687622, consensual act : and criminals
----------
0.23764611780643463, the pregnancy : and criminals
----------
0.23787079751491547, the child : and criminals
----------
0.23807649314403534, her attacker : and criminals
----------
0.23810160160064697, i pity : with hate
----------
0.23825782537460327, in woman : and criminals
----------
0.23843176662921906, this situation : and criminals
----------
0.23864218592643738, the subject : and criminals
----------
0.23879432678222656, who rights : and criminals
----------
0.2390087991952896, the woman : and criminals
----------
0.23916082084178925, the right : with hate
----------
0.23935815691947937, her attack : and criminals
----------
0.23949886858463287, attack yes : and criminals
----------
0.239670529961586, a right : and criminals
----------
0.23988282680511475, on compassion : and criminals
----------
0.24009570479393005, than morality : and criminals
----------
0.24032607674598694, or legality : and criminals
----------
0.2405395805835724, the fetus : are people
----------
0.24065327644348145, the right : the cause
----------
0.240862637758255, to life : and criminals
----------
0.24103814363479614, that life : and criminals
----------
0.24123099446296692, foster care : and criminals
----------
0.24144886434078217, adoptive family : and criminals
----------
0.24160698056221008, a right : with hate
----------
0.24180147051811218, both morality : and criminals
----------
0.24195964634418488, literal sense : and criminals
----------
0.2422015219926834, and legality : and criminals
----------
0.24239696562290192, ( murder : and criminals
----------
0.24262070655822754, a crime : and criminals
----------
0.2427913397550583, by jail-time : and criminals
----------
0.24300050735473633, even death : and criminals
----------
0.24315106868743896, the parents : and criminals
----------
0.24331846833229065, are teenagers : and criminals
----------
0.24356254935264587, a child : and criminals
----------
0.24379567801952362, the pregnancy : and criminals
----------
0.24396942555904388, trying time : and criminals
----------
0.24417994916439056, the ease : and criminals
----------
0.24437764286994934, two people : and criminals
----------
0.2445007562637329, people lives : and criminals
----------
0.24470330774784088, the continuation : and criminals
----------
0.24489985406398773, the child : are people
----------
0.24509471654891968, young parents : and criminals
----------
0.24525311589241028, exist care : and criminals
----------
0.24540305137634277, care options : and criminals
----------
0.24564136564731598, parent-less children : and criminals
----------
0.24570932984352112, the above : a hospital
----------
0.24584896862506866, that child : and criminals
----------
0.2460021674633026, child services : and criminals
----------
0.24609753489494324, services cost : and criminals
----------
0.24621644616127014, cost taxpayers : and criminals
----------
0.2463941127061844, ridiculous sum : and criminals
----------
0.24657054245471954, of money : and criminals
----------
0.24667051434516907, in response : with hate
----------
0.24674658477306366, the following : an infection
----------
0.24693180620670319, then people : and criminals
----------
0.24711234867572784, their sex : and criminals
----------
0.2472672015428543, sex drive : and criminals
----------
0.24746020138263702, immediate topic : and criminals
----------
0.24763396382331848, at hand : and criminals
----------
0.24779054522514343, should abortions : and criminals
----------
0.24792371690273285, my definition : are people
----------
0.24814336001873016, of life : and criminals
----------
0.24833226203918457, the start : are people
----------
0.2485349178314209, the heartbeat : are people
----------
0.24873878061771393, expecting woman : and criminals
----------
0.24891401827335358, four-week window : are people
----------
0.2490643709897995, an abortion : with hate
----------
0.249290332198143, the fetus : and criminals
----------
0.24942709505558014, two weeks : the cause
----------
0.24961690604686737, a pregnancy : are people
----------
0.2498239278793335, this woman : and criminals
----------
0.2500097453594208, two weeks : and lots
----------
0.25009000301361084, an abortion : of abortions
----------
0.2502148449420929, for pregnancy : with hate
----------
0.2503022253513336, a window : the cause
----------
ctn: 67
s_0
0.25032225251197815, like jonny : think abortion
----------
0.2503495216369629, jonny jacks : coat hanger
----------
0.2504183351993561, said abortion : of abortions
----------
0.25067687034606934, a fetus : and criminals
----------
0.2508983612060547, a baby : and criminals
----------
ctn: 68
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
0.2509283125400543, what punishment : think abortion
----------
0.25108206272125244, on abortionists : and criminals
----------
0.25116685032844543, tough question : the cause
----------
0.25126805901527405, a mixture : a question
----------
0.25137338042259216, reformative justice : and criminals
----------
0.2515011727809906, the retribution : with hate
----------
0.2515881359577179, suitable fines : of abortions
----------
0.25163450837135315, or jail : and criminals
----------
0.25168535113334656, jail sentences : are people
----------
0.2517111599445343, sentences reformation : coat hanger
----------
0.25180262327194214, the women : and criminals
----------
0.2518661916255951, why abortion : and criminals
----------
0.25194406509399414, the punishment : and criminals
----------
0.25204792618751526, these women : and criminals
----------
0.2521592378616333, killing humans : and criminals
----------
0.252160906791687, - banning : ) reproduction
----------
0.2521764934062958, the make : the cause
----------
0.2522624135017395, the practice : the people
----------
0.25242337584495544, the case : the cause
----------
0.2525227963924408, other crimes : with hate
----------
0.2525995075702667, as theft : and criminals
----------
0.2526581883430481, theft murder : of abortions
----------
0.2527139484882355, murder etc : and criminals
----------
0.2527586817741394, some abortions : of abortions
----------
0.25282761454582214, urestricted abortion : of abortions
----------
0.2529168725013733, significant increase : and criminals
----------
0.253054678440094, in abortions : and criminals
----------
0.2531171143054962, planned parenthood : of abortions
----------
0.25321048498153687, certain restrictions : of abortions
----------
0.2532983124256134, on abortion : of abortions
----------
0.25338929891586304, the number : are people
----------
0.2535950839519501, of abortions : with hate
----------
0.2536649703979492, my claim : the cause
----------
0.2537386119365692, banning abortions : with hate
----------
0.2538084089756012, some extent : and criminals
----------
0.2538497745990753, the numbers : the people
----------
0.2538517713546753, : http : ( eds
----------
0.2538963854312897, banning abortions : of abortions
----------
0.25398916006088257, moral obligation : with hate
----------
0.254130482673645, the mother : are people
----------
0.25418198108673096, mother and/or : and criminals
----------
0.25432905554771423, and/or child : and criminals
----------
0.2544490694999695, child life : and criminals
----------
0.25455597043037415, threatened abortion : and criminals
----------
0.2546731233596802, whole argument : and criminals
----------
0.25479453802108765, the right : with hate
----------
0.25495201349258423, to life : and criminals
----------
0.2550698518753052, second case : the cause
----------
0.25509798526763916, mercy killings : of abortions
----------
0.25513115525245667, the majority : the government
----------
0.25520142912864685, of abortions : of abortions
----------
0.25526225566864014, mercy killings : are people
----------
0.2552992105484009, that topic : and criminals
----------
0.25538620352745056, that men : and criminals
----------
0.2554427981376648, stop murder : are people
----------
0.25557324290275574, the babies : and criminals
----------
0.2556377053260803, stop slavery : with hate
----------
0.25569599866867065, not slave-owners : and criminals
----------
0.25583958625793457, the argument : the cause
----------
0.2559852600097656, the quality : are people
----------
0.2561388611793518, the argument : with hate
----------
0.256201833486557, the men : the people
----------
0.25627145171165466, ulterior motives : and criminals
----------
0.25641024112701416, that abortion : and criminals
----------
ctn: 69
s_0
0.25647464394569397, have sex : and criminals
----------
0.2565986216068268, without protection : and criminals
----------
ctn: 70
s_0
0.2566758692264557, bestows life : with hate
----------
0.256768137216568, bestows death : and criminals
----------
ctn: 71
s_0
s_1
s_2
0.25687432289123535, a person : are people
----------
0.2570243179798126, the child : are people
----------
0.2571042776107788, our technology : my gramps
----------
0.2571987807750702, these risks : and criminals
----------
0.2573367655277252, of death : and criminals
----------
0.2574350833892822, during childbirth : of abortions
----------
0.25748929381370544, some people : are people
----------
0.25752294063568115, full intention : the cause
----------
0.25764861702919006, a baby : a coat
----------
0.2577734887599945, during childbirth : with hate
----------
0.2578630745410919, are risks : and criminals
----------
0.25797876715660095, having children : are people
----------
0.2580391764640808, children period : the cause
----------
ctn: 72
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.25820034742355347, unborn child : and criminals
----------
0.25825920701026917, the world : the people
----------
0.25829845666885376, when someone : of people
----------
0.2583801746368408, the decision : the cause
----------
0.25856640934944153, a fetus : are people
----------
0.25874775648117065, own mother : and criminals
----------
0.25880226492881775, unborn children : are people
----------
0.25888505578041077, future hero : kill fetises
----------
0.25901591777801514, a solution : the cause
----------
0.25904905796051025, global warming : and criminals
----------
0.2591192126274109, of apathy : and criminals
----------
0.25924041867256165, to life : and criminals
----------
0.25927966833114624, can life : is murder
----------
0.2593285143375397, dead back : are people
----------
0.2594586908817291, to life : and criminals
----------
0.2595766484737396, the lives : the people
----------
0.2596684992313385, of everyone : and criminals
----------
0.2597719728946686, the unborn : are people
----------
0.25987640023231506, an abortion : of abortions
----------
0.2599581778049469, innocent term : and criminals
----------
0.2601395845413208, a life : are people
----------
ctn: 73
s_0
s_1
s_2
0.2602294981479645, an abortion : of abortions
----------
0.2604092061519623, a baby : and criminals
----------
0.26053720712661743, baby killer : and criminals
----------
0.2605522572994232, point blank : university press
----------
ctn: 74
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.2605748772621155, extreme poverty : and criminals
----------
0.2606470286846161, bringing children : and criminals
----------
0.2607853412628174, the world : and criminals
----------
0.26083117723464966, my parents : and criminals
----------
0.2608942985534668, a living : a coat
----------
0.26101768016815186, my dad : and criminals
----------
0.26108524203300476, a jazz : kill fetises
----------
0.2611338198184967, 2-3 gigs : the people
----------
0.26123443245887756, a week : the people
----------
0.26129475235939026, his studio : an infection
----------
0.2613186240196228, studio apt : an infection
----------
0.2613736391067505, and food : and criminals
----------
0.26147809624671936, my mom : and criminals
----------
0.26148733496665955, seamstress living : coat hanger
----------
0.26158198714256287, her parents : and criminals
----------
0.2616160809993744, my pops : her mie
----------
0.2617240250110626, his music : with hate
----------
0.26178961992263794, music career : is murder
----------
0.2618582248687744, a job : a question
----------
0.26191866397857666, the parks : the people
----------
0.26200899481773376, the city : the people
----------
0.2620409429073334, of pasadena : the people
----------
0.26214396953582764, a gardener : are people
----------
0.2621876895427704, my mom : the people
----------
0.26223987340927124, as newlyweds : and criminals
----------
0.2623266279697418, became parents : and criminals
----------
0.26236578822135925, 3 years : the cause
----------
0.26248466968536377, a sister : are people
----------
0.2626006007194519, a brother : are people
----------
0.2626307010650635, lower income : the child
----------
0.2627660036087036, income family : and criminals
----------
0.2628050744533539, my parents : her mie
----------
0.2628788948059082, paying jobs : with hate
----------
0.26291584968566895, as pre-teens : and criminals
----------
0.26297280192375183, middle class : and criminals
----------
0.26308226585388184, class home : and criminals
----------
0.2631685733795166, owning family : and criminals
----------
0.2633017897605896, the point : the cause
----------
0.26337599754333496, be income : are people
----------
0.2634631097316742, having children : and criminals
----------
0.26363909244537354, the motivation : with hate
----------
0.26378968358039856, one life : are people
----------
0.2639138400554657, the danger : the cause
----------
0.2639365792274475, having income : doing things
----------
0.2639453411102295, income requirement : birth control
----------
0.2640424072742462, bearing children : are people
----------
0.26414933800697327, the level : the cause
----------
0.264217734336853, the level : the cause
----------
0.2642763555049896, other factors : the cause
----------
0.26438358426094055, federal standards : the cause
----------
0.26439377665519714, big bbrother : and wolter
----------
0.26441076397895813, bbrother meddling : hair grws
----------
0.2645639479160309, our lives : and criminals
----------
0.2646263837814331, chinese model : the government
----------
0.2647090256214142, catholic model : are people
----------
0.264815092086792, preventive measure : are people
----------
0.2648760676383972, measure imo : are people
----------
0.2649327218532562, than government : and criminals
----------
0.26504039764404297, government regulation : and criminals
----------
0.2651565372943878, into child : are people
----------
0.26522254943847656, child bearing : and criminals
----------
ctn: 75
s_0
s_1
s_2
s_3
s_4
s_5
0.2653018534183502, same mother : are people
----------
0.26542264223098755, the baby : the people
----------
0.26548928022384644, the trash : with hate
----------
0.26554399728775024, take care : with hate
----------
0.265693336725235, of murder : and criminals
----------
0.26580536365509033, in jail : and criminals
----------
0.26596546173095703, the baby : are people
----------
0.2660501301288605, after birth : of abortions
----------
0.2662324607372284, a crime : and criminals
----------
0.26637670397758484, before birth : and criminals
----------
0.2665727138519287, a crime : and criminals
----------
0.2666921615600586, a house : of abortions
----------
0.2667732536792755, its arson : and criminals
----------
0.26691359281539917, same house : and criminals
----------
0.2669605016708374, half way : of abortions
----------
0.26702702045440674, its construction : the cause
----------
0.2670676112174988, still arson : the cause
----------
0.2671806216239929, a baby : is murder
----------
0.2673415541648865, a baby : are people
----------
0.26739802956581116, the moment : the people
----------
0.26748988032341003, its conceived : and lots
----------
ctn: 76
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.26762017607688904, against abortion : and criminals
----------
0.2676866054534912, this question : the cause
----------
0.2677513360977173, allow women : and criminals
----------
0.2678701877593994, receive abortions : with hate
----------
0.26796630024909973, a day : are people
----------
0.268162339925766, the child : are people
----------
0.268254816532135, most people : and criminals
----------
0.2682943344116211, a resounding : a question
----------
0.2684078514575958, this timeline : the cause
----------
0.2684970796108246, this fetus : this girl
----------
0.2686593234539032, human child : and criminals
----------
0.26879188418388367, the question : the cause
----------
0.2689174711704254, the woman : are people
----------
0.26894238591194153, woman right : are people
----------
0.26898103952407837, do something : are people
----------
0.26911649107933044, her body : are people
----------
0.2692156732082367, the risk : and lots
----------
0.2693265974521637, of pregnancy : with hate
----------
0.2694040536880493, for rape : with hate
----------
0.2694374918937683, sexual penetration : and criminals
----------
0.2694731652736664, her rights : a coat
----------
0.26953840255737305, the interests : with hate
----------
0.2696780562400818, the unborn : are people
----------
0.2697560489177704, the degree : the people
----------
0.26988404989242554, the principle : the people
----------
0.2698976397514343, principle stands : viability occurs
----------
0.26994237303733826, a side : a doctor
----------
0.26997312903404236, side note : 4th week
----------
0.2700410485267639, pro-choice scholars : and criminals
----------
0.27016720175743103, for abortion : with hate
----------
0.2702331244945526, abortion rights : and criminals
----------
0.2704092562198639, the child : are people
----------
0.2705492079257965, 1 year : are people
----------
0.27057695388793945, -- post : and criminals
----------
0.2706835865974426, post birth : are people
----------
ctn: 77
s_0
s_1
s_2
s_3
0.2707369923591614, with someone : and criminals
----------
0.2708209156990051, wrongful actions : and criminals
----------
0.2709314823150635, of others : and criminals
----------
ctn: 78
s_0
s_1
s_2
s_3
s_4
s_5
s_6
0.2710607945919037, the person : are people
----------
0.2712039351463318, your mother : are people
----------
0.27126604318618774, of pro-life : and criminals
----------
0.2714158892631531, kill life : and criminals
----------
0.2714964747428894, the possibilities : the people
----------
0.27157530188560486, the doors : the cause
----------
0.2716348171234131, lazy person : the people
----------
0.271780401468277, different situation : a question
----------
0.27184876799583435, if abortion : of abortions
----------
0.27202630043029785, a life : are people
----------
0.27209949493408203, a government : the people
----------
0.2721456289291382, government law : and criminals
----------
0.27227291464805603, about life : and criminals
----------
0.2723827064037323, and ethics : and criminals
----------
ctn: 79
s_0
s_1
s_2
s_3
0.2724792957305908, alright people : and criminals
----------
0.2726145386695862, another abortion : and criminals
----------
0.27269449830055237, abortion debate : and criminals
----------
0.27284371852874756, this topic : are people
----------
0.2728644013404846, apart spat : coat hanger
----------
0.27290451526641846, on crushed : and criminals
----------
0.2729954719543457, on convinceme : and criminals
----------
0.27307701110839844, 5 months : the cause
----------
0.2731519937515259, an abortion : of abortions
----------
0.2732066810131073, abortion debate : are people
----------
0.2732192575931549, debate please : are people
----------
0.2732222080230713, please wait : by scientists
----------
0.2733122408390045, a month : the cause
----------
0.2734614610671997, this topic : are people
----------
0.2735271751880646, every week : this time
----------
ctn: 80
s_0
s_1
0.2737104892730713, this debate : and criminals
----------
0.273768812417984, these conceptions : of abortions
----------
0.27388373017311096, the issuse : the cause
----------
ctn: 81
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
0.2739727199077606, every situation : the cause
----------
0.27405327558517456, situation abortion : are people
----------
0.27412399649620056, many people : and criminals
----------
0.27429214119911194, unborn baby : and criminals
----------
0.27448493242263794, a fetus : and criminals
----------
0.2746387720108032, human life : and criminals
----------
0.274680495262146, my opinion : her mie
----------
0.2747439742088318, the minute : the people
----------
0.27486172318458557, the sperm : of abortions
----------
0.2749271094799042, and egg : and lots
----------
0.2749497890472412, egg meet : coat hanger
----------
0.2751099169254303, the fetus : are people
----------
0.27524808049201965, human life : are people
----------
0.27540647983551025, a soul : and criminals
----------
0.2756364643573761, that child : and criminals
----------
0.27576664090156555, one situation : are people
----------
0.27588263154029846, think abortion : and criminals
----------
0.27606290578842163, necessary evil : and criminals
----------
0.27617761492729187, a woman : the people
----------
0.27633097767829895, the child : are people
----------
0.27637380361557007, 8 year : coat hanger
----------
0.27654576301574707, the child : and criminals
----------
0.276736319065094, only situation : with hate
----------
0.276878297328949, which abortion : and criminals
----------
0.27697405219078064, this life : an abortion
----------
0.2771085500717163, the consequences : and criminals
----------
0.27726855874061584, our actions : and criminals
----------
0.277357816696167, a teenage : this girl
----------
0.27747949957847595, teenage girl : and criminals
----------
0.2776690125465393, a child : and criminals
----------
0.2777651250362396, consensual sex : and criminals
----------
0.27793413400650024, that child : and criminals
----------
0.27808505296707153, give birth : with hate
----------
0.27834299206733704, that child : and criminals
----------
0.2784552276134491, that point : are people
----------
0.27862292528152466, the child : are people
----------
0.278733491897583, for adoption : and criminals
----------
0.2790153920650482, the child : and criminals
----------
0.27910682559013367, main point : a question
----------
0.2793002128601074, the mother : and criminals
----------
0.27951085567474365, innocent child : and criminals
----------
0.2795914113521576, all decisions : are people
----------
0.27972933650016785, after pregnancy : with hate
----------
0.279918372631073, the child : are people
----------
0.28009000420570374, his/her life : and criminals
----------
0.280125230550766, due respect : with hate
----------
0.2801479697227478, respect paige : is murder
----------
0.28019586205482483, paige jaclyn : and criminals
----------
ctn: 82
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
0.2803298830986023, the problem : the cause
----------
0.28045424818992615, this argument : with hate
----------
0.2805677056312561, the criteria : the people
----------
0.28071776032447815, a fetus : are people
----------
0.280883252620697, a fetus : are people
----------
0.28095903992652893, this statement : is murder
----------
0.28100740909576416, circular definition : the cause
----------
0.2810785174369812, your definition : the cause
----------
0.28110435605049133, of human : of people
----------
0.2811974585056305, second criteria : the cause
----------
0.2812372148036957, society fails : illigal try
----------
0.28133291006088257, a contribution : the cause
----------
0.2814115285873413, to society : and criminals
----------
0.2815420925617218, a fetus : a question
----------
0.28164973855018616, a matter : the cause
----------
0.2816775143146515, of time : of abortions
----------
0.281726598739624, sleeping person : a coat
----------
0.2817847430706024, this person : this girl
----------
0.2818646728992462, no contribution : with hate
----------
0.2820003926753998, a person : are people
----------
0.2821131646633148, the hospital : the people
----------
0.28213009238243103, what contribution : this view
----------
0.28216123580932617, the bedridden : this girl
----------
0.2822805941104889, sick person : are people
----------
0.2823992967605591, a contribution : of abortions
----------
0.28243041038513184, a developing : a coat
----------
0.2825852334499359, developing fetus : and criminals
----------
0.28277450799942017, joy-eliciting child : and criminals
----------
0.28285565972328186, your criteria : my gramps
----------
0.28294837474823, that abortion : of abortions
----------
0.28307297825813293, not murder : and criminals
----------
0.28317999839782715, stronger argument : with hate
----------
ctn: 83
s_0
s_1
s_2
0.28327125310897827, ignore womanhood : with hate
----------
0.2833094298839569, the right : an abortion
----------
0.2835116684436798, a person : and criminals
----------
0.283704936504364, the person : and criminals
----------
0.28387194871902466, the womb : and criminals
----------
0.2839333415031433, as nothing : and criminals
----------
ctn: 84
s_0
s_1
0.28401103615760803, make laws : and criminals
----------
0.2841230034828186, because people : and criminals
----------
0.28416749835014343, find ways : and criminals
----------
0.2842365503311157, around laws : with hate
----------
0.28437334299087524, our responsibility : with hate
----------
0.284483939409256, the laws : are people
----------
ctn: 85
s_0
s_1
0.2845773696899414, wrong thing : the cause
----------
0.2846733331680298, a persons : and criminals
----------
0.28483372926712036, persons choice : and criminals
----------
0.28499385714530945, their baby : and criminals
----------
0.285052627325058, legal parents : and criminals
----------
0.28517580032348633, their child : of abortions
----------
0.28536364436149597, the baby : and criminals
----------
0.2854979634284973, first place : and criminals
----------
0.2856128513813019, bad environment : the people
----------
0.28588539361953735, the child : and criminals
----------
0.2860248386859894, his gaurdians : and criminals
----------
0.28614139556884766, emotional mood : and criminals
----------
0.2863624691963196, their life : and criminals
----------
0.2863839864730835, and lead : and criminals
----------
0.2865143418312073, to others : and criminals
----------
ctn: 86
s_0
s_1
s_2
s_3
s_4
s_5
0.28662344813346863, a man : are people
----------
0.2866854965686798, a part : the cause
----------
0.28681713342666626, a woman : are people
----------
0.2869408130645752, the consequences : the people
----------
0.28700530529022217, having sex : with hate
----------
0.2870956361293793, without protection : with hate
----------
0.28710296750068665, dont use : are people
----------
0.2872084081172943, use protection : and criminals
----------
0.2872322201728821, well chance : the cause
----------
0.28743067383766174, a child : and criminals
----------
0.28745028376579285, common sense : fetal development
----------
0.2875898778438568, or woman : and criminals
----------
ctn: 87
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.2876058518886566, ok ive : illigal try
----------
0.28767362236976624, a fight : a question
----------
0.2877455949783325, with someone : and criminals
----------
0.28779375553131104, your arm : an infection
----------
0.28786131739616394, your arm : the cause
----------
0.28801751136779785, that fight : and criminals
----------
0.28818485140800476, sane person : and criminals
----------
0.288217157125473, person wouldnt : and criminals
----------
0.2882499694824219, if someone : illigal women
----------
0.2882830500602722, back alley : an infection
----------
0.2883720397949219, alley abortion : of abortions
----------
0.2886166572570801, that choice : and criminals
----------
0.28882619738578796, the results : and criminals
----------
0.28904202580451965, the risks : and criminals
----------
0.28914037346839905, is adoption : and criminals
----------
0.28914782404899597, dont forget : abortion controversy
----------
0.2891678214073181, abortion isnt : and lots
----------
0.2892186641693115, isnt right : with hate
----------
0.2893570363521576, other options : and criminals
----------
ctn: 88
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
0.28946298360824585, pro choice : and criminals
----------
0.2895512878894806, the subject : the people
----------
0.28957340121269226, my self : her self
----------
0.2896982729434967, my woman : and criminals
----------
0.289741575717926, right thing : the cause
----------
0.28976672887802124, and choose : and criminals
----------
0.28994572162628174, the child : and criminals
----------
0.28998059034347534, my right : her mie
----------
0.2899991571903229, so wat : is murder
----------
0.2900521755218506, as someone : and criminals
----------
0.29013732075691223, on someone : and criminals
----------
0.29029881954193115, elses life : and criminals
----------
0.29044386744499207, life decisions : and criminals
----------
0.290575236082077, the sin : the people
----------
0.2907065451145172, these people : and criminals
----------
0.29077109694480896, on judjment : and criminals
----------
0.29084324836730957, the subject : the cause
----------
0.2909602224826813, the choice : the cause
----------
0.29108381271362305, choice process : are people
----------
0.2912088632583618, the time : the cause
----------
0.29124489426612854, an abortion : of abortions
----------
0.29127174615859985, me nothing : are people
----------
0.2914145886898041, my kids : and criminals
----------
0.29143717885017395, and shes : and criminals
----------
0.29161685705184937, my body : and criminals
----------
ctn: 89
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
0.2917673885822296, a doctor : and criminals
----------
0.29183900356292725, two mothers : are people
----------
0.29184409976005554, both ask : two individuals
----------
0.29195401072502136, your choice : the cause
----------
0.29201948642730713, an abortion : of abortions
----------
0.2921150028705597, the weight : the cause
----------
0.29223501682281494, a life : a question
----------
0.2923300266265869, your shoulders : with hate
----------
0.29245445132255554, the backgrounds : the people
----------
0.2925778329372406, the mothers : and criminals
----------
0.2926739454269409, the statement : the people
----------
0.2927674949169159, the world : are people
----------
0.2928248941898346, beautiful mind : are people
----------
0.29283177852630615, da vinci : mie twenties
----------
0.2929679751396179, the cruelty : the people
----------
0.2929883897304535, of adolf : and criminals
----------
0.29304903745651245, adolf hitler : and criminals
----------
0.2931913435459137, every child : and criminals
----------
0.2933316230773926, this world : and criminals
----------
0.29334619641304016, few extreme : with hate
----------
0.29344606399536133, extreme circumstances : and criminals
----------
0.29356157779693604, unwanted pregnancy : of abortions
----------
0.2936978042125702, the pregnancy : of abortions
----------
0.2937805950641632, early stages : are people
----------
0.29386797547340393, my view : a question
----------
ctn: 90
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
0.29394087195396423, many times : and criminals
----------
0.294046014547348, this subject : and criminals
----------
0.29414841532707214, this debate : and criminals
----------
0.29424476623535156, a women : the people
----------
0.2943534255027771, fertilized egg : of abortions
----------
0.29453936219215393, a choice : with hate
----------
0.2947635054588318, the argument : and criminals
----------
0.2948981523513794, about abortion : and criminals
----------
0.2949824929237366, in abortion : of abortions
----------
0.295166939496994, a choice : with hate
----------
0.29520583152770996, choice wheather : are people
----------
0.2952255606651306, ligitamet proof : kennedy inst
----------
0.29532015323638916, its mother : an infection
----------
0.29535821080207825, mother isnt : and lots
----------
0.2954813241958618, this debate : and criminals
----------
0.29556962847709656, and nobody : and criminals
----------
0.2956840395927429, an abortion : with hate
----------
0.29590606689453125, my choice : and criminals
----------
0.2959839105606079, choice wheather : with hate
----------
0.295992374420166, just sayin : my gramps
----------
0.29611027240753174, my beliefs : with hate
----------
0.2961955964565277, are abortion : with hate
----------
0.2964077889919281, a baby : with hate
----------
0.29650965332984924, my beliefs : with hate
----------
0.2966318428516388, my beliefs : with hate
----------
0.29667970538139343, have mine : and criminals
----------
0.29678529500961304, wheather anyone : and criminals
----------
ctn: 91
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
0.2968733608722687, your question : the cause
----------
0.29693418741226196, at conception : of abortions
----------
0.2970612645149231, of religion : and criminals
----------
0.2971678078174591, of science : and criminals
----------
0.29722315073013306, at conception : of abortions
----------
0.29738450050354004, the embryo : of abortions
----------
0.2974013388156891, the constituent : as birth
----------
0.29744523763656616, constituent parts : and lots
----------
0.2975924611091614, for growth : with hate
----------
0.29775407910346985, and maturation : and criminals
----------
0.29783913493156433, same things : a question
----------
0.2979329228401184, : food : with hate
----------
0.29797595739364624, food oxygen : an infection
----------
0.2979985475540161, oxygen water : selves abortions
----------
0.2981073558330536, and shelter : of abortions
----------
0.29817840456962585, only difference : a question
----------
0.29830387234687805, the means : the people
----------
0.2984931170940399, the child : are people
----------
0.2987198233604431, the child : and criminals
----------
0.2988417446613312, a parapalegic : coat hanger
----------
0.29887697100639343, get food/water/oxygen/shelter : are people
----------
0.2989887595176697, for his/herself : and criminals
----------
0.29902082681655884, christopher reeve : coat hanger
----------
0.2990577816963196, his accident : is murder
----------
0.2991183400154114, no one : and criminals
----------
0.29915565252304077, legal definition : the cause
----------
0.2991921305656433, supreme court : the government
----------
0.2992645800113678, court justice : coat hanger
----------
0.299299955368042, justice harry : illigal try
----------
0.2994022071361542, harry blackmun : are people
----------
0.2994711101055145, medical background : the cause
----------
0.2995988130569458, the trimester : the people
----------
0.29966938495635986, trimester system : the people
----------
ctn: 92
s_0
s_1
s_2
s_3
s_4
0.2998153567314148, the debate : of abortions
----------
0.29991158843040466, the problem : the cause
----------
0.30001652240753174, of consent : with hate
----------
0.30024516582489014, the foetus : and criminals
----------
0.30046409368515015, a person : and criminals
----------
0.30057403445243835, person question : and criminals
----------
0.3006384074687958, - philosophers : and criminals
----------
0.3006702661514282, making law : with hate
----------
0.3007691502571106, to practicallity : and criminals
----------
0.3008193075656891, much liklihood : and criminals
----------
0.3008836507797241, therefore lawmakers : and criminals
----------
0.3010406792163849, a person : and criminals
----------
0.3011511564254761, moral qualities : and criminals
----------
0.30132055282592773, a person : and criminals
----------
0.30147427320480347, person life : and criminals
----------
0.3016156554222107, the liberty : the cause
----------
0.30183297395706177, the woman : are people
----------
0.30199918150901794, her body : are people
----------
0.3020940124988556, important right : are people
----------
0.30227550864219666, to life : and criminals
----------
0.3023929297924042, in danger : with hate
----------
ctn: 93
s_0
0.3025527894496918, your kidney : of abortions
----------
ctn: 94
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
0.30268436670303345, a comment : and criminals
----------
0.30278831720352173, nay column : and criminals
----------
0.3029645085334778, nay/yay vote : and criminals
----------
0.30308377742767334, vote lol : and criminals
----------
0.3032476305961609, a woman : and criminals
----------
0.3033409118652344, the right : with hate
----------
0.30340686440467834, with caveats : with hate
----------
0.3034747540950775, a time : a question
----------
0.3036310374736786, the woman : are people
----------
0.3036932945251465, a living : a coat
----------
0.303742378950119, the day : the people
----------
0.3038312792778015, the baby : an infection
----------
0.3039631247520447, a baby : are people
----------
0.30407246947288513, the mother : the people
----------
0.30413371324539185, in labor : with hate
----------
0.30425259470939636, her mind : the cause
----------
0.30433225631713867, unrealistic examples : and criminals
----------
0.30444151163101196, the point : the cause
----------
0.30458852648735046, a point : of abortions
----------
0.30478227138519287, the woman : and criminals
----------
0.30486735701560974, clear murder : and criminals
----------
0.30503049492836, that point : and criminals
----------
0.30517178773880005, the woman : are people
----------
0.30522313714027405, the right : an infection
----------
0.3054417371749878, the man : and criminals
----------
0.30560562014579773, the man : and criminals
----------
0.30577710270881653, the woman : and criminals
----------
0.3059120774269104, an abortion : and criminals
----------
0.3059600293636322, however abortion : of abortions
----------
0.3060832917690277, the result : are people
----------
0.3062354326248169, a woman : and criminals
----------
0.30644044280052185, a mother : and criminals
----------
0.3064863681793213, okay fine : illigal try
----------
0.30665522813796997, the man : and criminals
----------
0.3068050146102905, same choice : with hate
----------
0.3068878650665283, particular point : a question
----------
0.30699124932289124, the fact : the cause
----------
0.3070722818374634, pregnant one : are people
----------
0.3071259558200836, the reason : the cause
----------
0.30725714564323425, the choice : of abortions
----------
0.3073956072330475, the point : the cause
----------
0.30754631757736206, a mother : are people
----------
0.30775323510169983, the man : and criminals
----------
0.30787453055381775, same right : and criminals
----------
0.3080906271934509, a father : and criminals
----------
0.3081999123096466, the baby : an infection
----------
0.3083750903606415, his connection : are people
----------
ctn: 95
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
s_12
s_13
s_14
s_15
s_16
s_17
s_18
s_19
s_20
s_21
s_22
s_23
s_24
s_25
s_26
s_27
s_28
s_29
s_30
s_31
s_32
s_33
s_34
s_35
s_36
s_37
s_38
s_39
s_40
s_41
s_42
s_43
s_44
0.30850666761398315, the issue : the cause
----------
0.3086264431476593, of abortion : with hate
----------
0.30871909856796265, many ways : with hate
----------
0.3087456226348877, pro-life supporters : and criminals
----------
0.30882471799850464, use religion : and criminals
----------
0.3088608384132385, religion claim : and criminals
----------
0.3089677691459656, that life : and criminals
----------
0.3090740740299225, at conception : and criminals
----------
0.3091585040092468, a fit : and lots
----------
0.3092534840106964, any sort : are people
----------
0.30944404006004333, of abortion : and criminals
----------
0.30959582328796387, as murder : and criminals
----------
0.30965912342071533, the developing : are people
----------
0.3098435401916504, developing fetus : and criminals
----------
0.3098652958869934, pro-choice supporters : are people
----------
0.30987969040870667, hear anything : with hate
----------
0.3099070191383362, other side : are people
----------
0.31005334854125977, a woman : and criminals
----------
0.31025230884552, her body : and criminals
----------
0.310417503118515, own business : and criminals
----------
0.3105611801147461, say religion : and criminals
----------
0.31067168712615967, no place : and criminals
----------
0.3108765482902527, the debate : and criminals
----------
0.3110992908477783, the issue : and criminals
----------
0.31120726466178894, the truth : with hate
----------
0.3112297058105469, big time : a life
----------
0.31134092807769775, time supporter : and criminals
----------
0.3114616572856903, pro-choice movement : and criminals
----------
0.31154143810272217, standard arguments : the cause
----------
0.31171566247940063, the government : and criminals
----------
0.3118600845336914, a woman : and criminals
----------
0.3120446801185608, her body : with hate
----------
0.3121432363986969, thought process : are people
----------
0.312204509973526, a bit : a coat
----------
0.3123447895050049, pro-life supporter : and criminals
----------
0.3124414086341858, my reasons : with hate
----------
0.31255945563316345, my thinking : and criminals
----------
0.3127675950527191, the issue : and criminals
----------
0.3128741979598999, correct view : are people
----------
0.3129590153694153, view point : are people
----------
0.31312763690948486, the issue : with hate
----------
0.3132365345954895, my opinion : with hate
----------
0.3133481442928314, support abortion : with hate
----------
0.3135738968849182, the woman : and criminals
----------
0.31368350982666016, woman life : and criminals
----------
0.31379640102386475, in danger : and criminals
----------
0.3140280246734619, the child : and criminals
----------
0.3142632842063904, her trauma : and criminals
----------
0.31444114446640015, child quality : and criminals
----------
0.31469011306762695, of life : and criminals
----------
0.3147660493850708, consider abortion : with hate
----------
0.314911425113678, a couple : are people
----------
0.3151150941848755, their child : and criminals
----------
0.3153395652770996, meaningful life : and criminals
----------
0.3154906928539276, support abortions : and criminals
----------
0.315630704164505, of convenience : and criminals
----------
0.315714567899704, ample methods : the cause
----------
0.315818727016449, of contraception : and lots
----------
0.31595665216445923, from condoms : with hate
----------
0.3160552382469177, birth control : and criminals
----------
0.3161869943141937, control pills : and criminals
----------
0.31629493832588196, these methods : the people
----------
0.3163850009441376, low prices : and criminals
----------
0.3163893222808838, the morning : the time
----------
0.3164205849170685, after pill : an infection
----------
0.316457062959671, an accident : is murder
----------
0.316484659910202, i support : a story
----------
0.3165859580039978, the time : the cause
----------
0.3166483938694, time frame : the cause
----------
0.3166850507259369, 72 hours : mie twenties
----------
0.316711962223053, hold society : kill fetises
----------
0.316814661026001, the choices : and criminals
----------
0.316938579082489, a woman : are people
----------
0.3170049786567688, having sex : with hate
----------
0.3170362412929535, i knew : are people
----------
0.31719648838043213, in pregnancy : and criminals
----------
0.317351371049881, this baby : and criminals
----------
0.31750214099884033, a nuisance : and criminals
----------
0.31773242354393005, the pregnancy : and criminals
----------
0.31793975830078125, the fetus : and criminals
----------
0.31808385252952576, an object : and criminals
----------
0.31816715002059937, the fact : the people
----------
0.3183651566505432, a child : and criminals
----------
0.31850504875183105, a piece : are people
----------
0.3186052739620209, of garbage : and criminals
----------
0.31862765550613403, one percent : a fetus
----------
0.31869789958000183, of abortions : with hate
----------
0.3187906742095947, of rape : and criminals
----------
0.31887704133987427, or incest : and criminals
----------
0.318903386592865, six percent : are people
----------
0.31900137662887573, of abortions : with hate
----------
0.3190253973007202, medical reasons : with hate
----------
0.31903791427612305, 93 percent : and lots
----------
0.31912580132484436, of abortions : and criminals
----------
0.3191753327846527, social reasons : and criminals
----------
0.31919074058532715, for example : are people
----------
0.3193674087524414, the child : are people
----------
0.319536954164505, the child : and criminals
----------
0.3197207748889923, the mother : and criminals
----------
0.3198498487472534, mother life : and criminals
----------
0.3198709189891815, by week : about weather
----------
0.31996750831604004, a baby : is murder
----------
0.32004639506340027, baby heart : an infection
----------
0.32006826996803284, and blood : of infection
----------
0.32020604610443115, the body : of abortions
----------
0.3203390836715698, the womb : the cause
----------
0.3203789293766022, if someone : of people
----------
0.3204217255115509, took actions : with hate
----------
0.3205554783344269, another person : and criminals
----------
0.32059264183044434, person heart : an infection
----------
0.3207457661628723, them murderers : and criminals
----------
0.3208836019039154, the person : and criminals
----------
0.321050226688385, the womb : are people
----------
0.3211148977279663, fair game : of abortions
----------
0.3211851119995117, all abortions : with hate
----------
0.3212655484676361, 9th week : are people
----------
0.32141655683517456, of pregnancy : with hate
----------
0.3215051591396332, % happen : and criminals
----------
0.3215869963169098, 10th week : are people
----------
0.32169023156166077, % happen : are people
----------
0.32177865505218506, 12th week : are people
----------
0.32186996936798096, % happen : and criminals
----------
0.3219585120677948, 15th week : are people
----------
0.32205531001091003, % happen : with hate
----------
0.3221222460269928, 20th week : are people
----------
0.3222131133079529, all abortions : with hate
----------
0.3222362697124481, ) happen : ) reproduction
----------
0.3223126232624054, 20th week : the cause
----------
0.3224601745605469, of pregnancy : of abortions
----------
0.32251259684562683, the majority : the people
----------
0.32261866331100464, of abortions : and criminals
----------
0.3227885663509369, ninth week : and criminals
----------
0.3229524791240692, of pregnancy : with hate
----------
0.32305654883384705, the heart : the people
----------
0.3231861889362335, sixth week : are people
----------
0.3232839107513428, the abortion : of abortions
----------
0.32331719994544983, takes place : are people
----------
0.32343602180480957, sixth week : are people
----------
0.3235558271408081, invasive action : with hate
----------
0.3236543536186218, the heart : of abortions
----------
0.3236843943595886, heart beat : of abortions
----------
0.3237403333187103, further development : and lots
----------
0.3238617479801178, viable pregnancy : of abortions
----------
0.3238828182220459, by week : about weather
----------
0.3240072727203369, the baby : the people
----------
0.324194073677063, a fetus : are people
----------
0.3242703080177307, the statistics : are people
----------
0.324290931224823, 12 percent : the cause
----------
0.32436907291412354, of abortions : with hate
----------
0.3243965208530426, the eleventh : the cause
----------
0.3244883120059967, twelfth weeks : the people
----------
0.32462775707244873, of pregnancy : with hate
----------
0.3247651159763336, the stages : the people
----------
0.32489874958992004, of development : are people
----------
0.32490378618240356, development let : illigal try
----------
0.3249073028564453, let look : biology looks
----------
0.32504168152809143, at contraception : with hate
----------
0.3251452147960663, contraception use : with hate
----------
0.3252531588077545, an abortion : with hate
----------
0.32526254653930664, fifty-four percent : 25 weeks
----------
0.3253072202205658, of women : the people
----------
0.3253662586212158, have abortions : of abortions
----------
0.3253995478153229, contraceptive method : the cause
----------
0.32547467947006226, the condom : the people
----------
0.3255663514137268, the pill : and lots
----------
0.32565972208976746, the month : are people
----------
0.32570746541023254, those women : are people
----------
0.3257203698158264, of pill : of infection
----------
0.3257618546485901, pill users : and criminals
----------
0.3257773220539093, of condom : as birth
----------
0.32581573724746704, condom users : and criminals
----------
0.32596471905708313, their method : with hate
----------
0.3259761333465576, of pill : as birth
----------
0.3259930908679962, pill users : the people
----------
0.3260032534599304, of condom : as birth
----------
0.3260214626789093, condom users : are people
----------
0.3260464370250702, correct use : the cause
----------
0.32606804370880127, of pill : of infection
----------
0.32609328627586365, pill users : and criminals
----------
0.3261616826057434, an abortion : of abortions
----------
0.32617056369781494, abortion report : illigal try
----------
0.3262336850166321, used birth : of abortions
----------
0.32627320289611816, birth control : the cause
----------
0.32638272643089294, the month : are people
----------
0.32641008496284485, of condom : with hate
----------
0.32643434405326843, condom users : are people
----------
0.32648611068725586, used condoms : with hate
----------
0.3265897035598755, the month : are people
----------
0.32669028639793396, the responsibility : the cause
----------
0.326805055141449, a method : the cause
----------
0.3268972635269165, of birth : of abortions
----------
0.3269664943218231, birth control : and lots
----------
0.3270381689071655, have sex : with hate
----------
0.3271474540233612, your pregnancy : the people
----------
0.3273000419139862, the time : with hate
----------
0.32732734084129333, of pill : of infection
----------
0.3273559808731079, pill users : and criminals
----------
0.32737091183662415, of condom : use abortion
----------
0.32740145921707153, condom users : are people
----------
0.3274751305580139, their birth : an abortion
----------
0.3275053799152374, birth control : proper protection
----------
0.32754549384117126, control method : are people
----------
0.32758763432502747, high number : the cause
----------
0.32762235403060913, previous numbers : the cause
----------
0.32768580317497253, have sex : with hate
----------
0.3277706503868103, use protection : with hate
----------
0.327811598777771, protection good : and lots
----------
0.3279593288898468, semi-responsible decision : and criminals
----------
0.3279707431793213, 100 percent : 25 weeks
----------
0.3280605375766754, percent protection : with hate
----------
0.3281525671482086, besides abstinence : and criminals
----------
0.3282269835472107, had sex : with hate
----------
0.3283459544181824, the consequences : the people
----------
0.32848748564720154, your actions : and criminals
----------
0.3285648822784424, an adult : and criminals
----------
0.32856789231300354, let look : biology looks
----------
0.3286832273006439, this way : are people
----------
0.328773558139801, allow women : and criminals
----------
0.3289543688297272, unborn children : and criminals
----------
0.32910677790641785, as inconveniences : and criminals
----------
0.32916930317878723, inconveniences obstacles : and criminals
----------
0.3293043076992035, unwanted additions : and criminals
----------
0.3294418752193451, a woman : are people
----------
0.3295813500881195, a child : are people
----------
0.3297559916973114, a burden : and criminals
----------
0.3299051523208618, a woman : with hate
----------
0.3300692141056061, her toddler : with hate
----------
0.3302193284034729, toddler child : and criminals
----------
0.3303738534450531, or teenager : and criminals
----------
0.3304334580898285, the difference : the cause
----------
0.3304901719093323, of abortion : of abortions
----------
0.3305882215499878, that abortion : and criminals
----------
0.33069032430648804, not murder : and criminals
----------
0.33087101578712463, the fetus : are people
----------
0.33108747005462646, the womb : and criminals
----------
0.3312305212020874, the time : with hate
----------
0.33141589164733887, the abortion : with hate
----------
0.3315628468990326, the case : with hate
----------
0.3316742181777954, the case : the cause
----------
0.33179110288619995, every woman : and criminals
----------
0.33197370171546936, her pregnancy : are people
----------
0.33214274048805237, the fetus : are people
----------
0.33225464820861816, the fact : are people
----------
0.33229729533195496, the road : her mie
----------
0.33248448371887207, the fetus : are people
----------
0.33253034949302673, to term : and criminals
----------
0.33268606662750244, alive baby : are people
----------
0.33284562826156616, the child : the people
----------
0.3329422175884247, child experience : and criminals
----------
0.33304938673973083, the labor : the cause
----------
0.33314311504364014, and delivery : and criminals
----------
0.333230197429657, the inconvenience : the people
----------
0.3334391117095947, the child : and criminals
----------
0.33363616466522217, your actions : and criminals
----------
0.33378809690475464, the baby : the people
----------
0.3339473307132721, for adoption : and criminals
----------
0.33396920561790466, are plenty : are people
----------
0.33404651284217834, of couples : and criminals
----------
0.3342682123184204, the child : and criminals
----------
0.33431026339530945, medical waste : and criminals
----------
0.3343583643436432, waste container : and criminals
----------
ctn: 96
s_0
0.3344144821166992, worst form : the cause
----------
0.334480345249176, of child : of abortions
----------
0.33457186818122864, child abuse : and criminals
----------
ctn: 97
s_0
s_1
0.33469897508621216, a child : are people
----------
0.33481261134147644, the age : are people
----------
0.3348419964313507, throws temper-tantrums : with hate
----------
0.3350014090538025, a child : are people
----------
0.3350287675857544, throws temper-tantrums : of abortions
----------
ctn: 98
s_0
s_1
s_2
s_3
s_4
s_5
0.33504340052604675, 16 days : 4th week
----------
0.3350924849510193, days post-ovulation : this girl
----------
0.3352302014827728, the baby : the people
----------
0.33529359102249146, baby brain : of infection
----------
0.3354257643222809, 20 weeks : of abortions
----------
0.3355320692062378, weeks respond : of abortions
----------
0.3356914520263672, painful procedures : and criminals
----------
0.3357510268688202, correct things : with hate
----------
0.33590611815452576, the baby : the people
----------
0.336089164018631, the uterus : are people
----------
0.3361465334892273, elevated heart : the cause
----------
0.33619424700737, heart rate : and lots
----------
0.33627772331237793, and secretion : and criminals
----------
0.33636465668678284, of stress : with hate
----------
0.3363907039165497, stress hormones : coat hanger
----------
0.336587518453598, the fetus : the people
----------
0.33664193749427795, bjorn merker : kill fetises
----------
0.33676382899284363, swedish neuroscientist : the people
----------
0.3367786109447479, extensive research : a story
----------
0.3368953466415405, on children : are people
----------
0.33698365092277527, developed cortexes : with hate
----------
0.3370584547519684, the cortex : the cause
----------
0.3371486961841583, the perception : the people
----------
0.3372778296470642, of pain : with hate
----------
0.33730682730674744, tacit consensus : basic books
----------
0.3373613655567169, cerebral cortex : the cause
----------
0.3374633193016052, the organ : an infection
----------
0.33753710985183716, of consciousness : of abortions
----------
0.3375525176525116, ( information : a story
----------
0.3375657796859741, cq researcher : san fransisco
----------
ctn: 99
s_0
s_1
0.33760857582092285, white males : and criminals
----------
0.3377250134944916, against abortions : and criminals
----------
0.33778971433639526, my friends : and criminals
----------
0.3378511667251587, at school : and criminals
----------
0.33800292015075684, are females : and criminals
----------
0.3381425440311432, abhor abortion : and criminals
----------
0.3382996618747711, is murder : and criminals
----------
0.3383916914463043, one sex : with hate
----------
0.3384246230125427, or race : and criminals
----------
0.33847296237945557, doing/saying something : are people
----------
ctn: 100
s_0
s_1
s_2
s_3
0.3385518491268158, for everyone : are people
----------
0.3386716842651367, about rape : with hate
----------
0.33872535824775696, one thing : a question
----------
0.33877426385879517, huge fight : and criminals
----------
0.33882036805152893, with someone : and criminals
----------
0.33886247873306274, your hand : the cause
----------
0.3388994634151459, or arm : the cause
----------
0.3390854299068451, that expirience : and criminals
----------
0.3390980660915375, actualy sane : the unborn
----------
0.3392457067966461, a physiatrist : are people
----------
ctn: 101
s_0
s_1
s_2
s_3
0.33933061361312866, the government : the people
----------
0.3393610715866089, no right : one point
----------
0.3394432067871094, allow murder : and criminals
----------
0.33958160877227783, a woman : are people
----------
0.339756041765213, a child : are people
----------
0.339873343706131, her boyfriend : and criminals
----------
0.3399423658847809, or husband : and criminals
----------
0.3399554491043091, or whoever : because someone
----------
0.34016624093055725, the responsibility : and criminals
----------
0.34040766954421997, a child : and criminals
----------
0.34047290682792664, no matter : with hate
----------
0.3406020998954773, kill someone : and criminals
----------
0.34064623713493347, more care : with hate
----------
0.34077349305152893, the parents : are people
----------
0.3408891260623932, the child : the cause
----------
0.34106218814849854, for adoption : with hate
----------
0.34119078516960144, foster care : and criminals
----------
ctn: 102
s_0
s_1
0.3412855565547943, my opinion : a question
----------
0.34144648909568787, the child : and criminals
----------
0.3415379524230957, for adoption : with hate
----------
0.3416355550289154, an abortion : with hate
----------
0.34183505177497864, the child : and criminals
----------
0.3419637084007263, for adoption : and criminals
----------
ctn: 103
s_0
0.34208303689956665, a decision : of abortions
----------
0.34221193194389343, the world : are people
----------
0.34222450852394104, on cant : with hate
----------
0.3423856198787689, one thing : and criminals
----------
0.3424988389015198, make decisions : and criminals
----------
0.34251585602760315, only effect : think abortion
----------
0.3426605761051178, other people : and criminals
----------
0.34278637170791626, different ball : and criminals
----------
0.3428846001625061, ball game : and criminals
----------
ctn: 104
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
s_11
0.3429579734802246, a catholic : are people
----------
0.34306731820106506, my religion : with hate
----------
0.34323015809059143, innocent babies : and criminals
----------
0.34329769015312195, catholic church : mie twenties
----------
0.34337323904037476, the egg : the cause
----------
0.34351715445518494, a baby : are people
----------
0.3437005281448364, a baby : are people
----------
0.3439154326915741, a choice : with hate
----------
0.3440202474594116, other religions : the government
----------
0.34420016407966614, your religion : with hate
----------
0.3443574607372284, your baby : are people
----------
0.34444400668144226, for adoption : with hate
----------
0.3445869982242584, the government : and criminals
----------
0.3446337580680847, more money : and criminals
----------
0.3447241187095642, into adoption : with hate
----------
0.34482935070991516, adoption agencies : and criminals
----------
0.34491702914237976, into seniors : and criminals
----------
0.34498903155326843, that seniors : and criminals
----------
0.34515854716300964, our lives : and criminals
----------
0.34540361166000366, the children : and criminals
----------
0.345466285943985, our future : the cause
----------
0.34550562500953674, if someone : of people
----------
0.3456699550151825, the baby : are people
----------
0.3458598256111145, a baby : and criminals
----------
0.34594082832336426, my opinion~ : her mie
----------
ctn: 105
s_0
s_1
s_2
0.34598037600517273, actually rani15 : of abortions
----------
0.34609851241111755, innocent life : and criminals
----------
0.3461949825286865, the death : the cause
----------
0.34622034430503845, death penalty : coat hanger
----------
0.3462637960910797, the taking : the cause
----------
0.34641021490097046, guilty life : and criminals
----------
0.3465312719345093, innocent one : and criminals
----------
ctn: 106
s_0
s_1
s_2
s_3
s_4
s_5
s_6
s_7
s_8
s_9
s_10
0.34657779335975647, the fact : the cause
----------
0.3466733396053314, that abortion : with hate
----------
0.3468714952468872, a person : and criminals
----------
0.3470005989074707, the fetus : the cause
----------
0.347087562084198, next state : are people
----------
0.3473205864429474, the victim : and criminals
----------
0.3474368155002594, victim future : and criminals
----------
0.34759002923965454, unborn baby : are people
----------
0.3476601541042328, an array : the cause
----------
0.34772753715515137, potential life : the cause
----------
0.34777602553367615, life paths : coat hanger
----------
0.3478442132472992, in front : and lots
----------
0.34800106287002563, your life : are people
----------
0.34806087613105774, unwanted children : the people
----------
0.3480736315250397, children thing : fetus bipedal
----------
0.34810271859169006, makes sense : with hate
----------
0.3482174277305603, the value : are people
----------
0.3483210504055023, of life : and criminals
----------
0.3483925461769104, into statistics : and criminals
----------
0.34846946597099304, less people : and criminals
----------
0.3485710918903351, less crime : and criminals
----------
0.3486269414424896, that alot : and criminals
----------
0.34869250655174255, of people : and criminals
----------
0.34874844551086426, their lives : the people
----------
0.34890681505203247, unwanted children : and criminals
----------
0.34899958968162537, great things : and criminals
----------
0.34913694858551025, humane solutions : are people
----------
0.3492559492588043, the crime : and criminals
----------
0.3493415117263794, crime problem : and criminals
----------
0.349383682012558, potential mothers : and criminals
----------
0.34956803917884827, the adoption : with hate
----------
0.34973347187042236, adoption process : are people
----------
0.3499470055103302, the children : and criminals
----------
0.3499705493450165, killing tha : of abortions
----------
0.3501436114311218, tha babies : and criminals
----------
ctn: 107
s_0
s_1
s_2
s_3
s_4
s_5
0.35019227862358093, the government : the people
----------
0.3502771854400635, the government : the people
----------
0.3503003716468811, recognize blacks : are people
----------
0.3503899872303009, being people : and criminals
----------
0.3504796326160431, or women : and criminals
----------
0.3505551815032959, having rights : and criminals
----------
0.35064008831977844, the government : the people
----------
0.3507300615310669, black person : of people
----------
0.35092705488204956, a person : and criminals
----------
0.351032018661499, but property : and criminals
----------
0.35113325715065, several people : and criminals
----------
0.3512546718120575, my argument : and criminals
----------
0.35132265090942383, early government : and criminals
----------
0.3514324724674225, that way : and criminals
----------
0.35153284668922424, no water : and criminals
----------
0.35158443450927734, federal government : the people
----------
0.3516543209552765, penalize murder : and criminals
----------
0.3517155647277832, make abortion : with hate
----------
0.3517551124095917, abortion right : are people
----------
0.35178911685943604, a human : a question
----------
0.3519749641418457, a fetus : are people
----------
0.35204771161079407, for evidence : with hate
----------
0.3520643711090088, evidence look : and lots
----------
0.3521241247653961, my longer : her self
----------
0.3522515296936035, longer argument : the cause
----------
ctn: 108
s_0
s_1
s_2
s_3
s_4
s_5
0.3523246943950653, spin facts : with hate
----------
0.35233092308044434, make ourselves : make abortions
----------
0.3524555265903473, the situations : are people
----------
0.3525388538837433, the case : the cause
----------
0.3526357114315033, of abortion : of abortions
----------
0.3527872860431671, the baby : the people
----------
0.35288316011428833, human form : are people
----------
0.3529757559299469, its existence : and criminals
----------
0.35303255915641785, this line : a coat
----------
0.3530642092227936, of thought : fetal development
----------
0.3531107008457184, an apple : is murder
----------
0.35315459966659546, apple seed : coat hanger
----------
0.35328662395477295, edible apple : are people
----------
0.3534392714500427, a farmer : are people
----------
0.3534967601299286, farmer stock : coat hanger
----------
0.353593647480011, of seed : with hate
----------
0.3536849617958069, final product : the people
----------
0.3538590669631958, his livelihood : and criminals
----------
0.3539186120033264, livelihood regardless : are people
----------
0.3540557026863098, a fetus : a question
----------
ctn: 109
s_0
s_1
s_2
s_3
s_4
s_5
0.3540932536125183, no sense : with hate
----------
0.3541524410247803, a fight : doing things
----------
0.35419613122940063, with someone : and criminals
----------
0.35422536730766296, your arm : an infection
----------
0.3542327880859375, arm broken : an infection
----------
0.35424819588661194, or something : doing things
----------
0.35434776544570923, called burdensum : and criminals
----------
0.3544040024280548, your sane : the cause
----------
0.3545222580432892, the difference : with hate
----------
0.3545387387275696, is nothing : and lots
----------
0.3545568585395813, nothing burdensum : are people
----------
0.3546930253505707, a baby : are people
----------
0.3548614978790283, the life : are people
----------
0.35488757491111755, the heck : the people
----------
0.3549332320690155, your arms : the cause
----------
0.35494595766067505, intact hows : illigal try
----------
0.3549899160861969, it fare : illigal try
----------
ctn: 110
s_0
s_1
s_2
s_3
s_4
0.35505571961402893, -think- abortion : of abortions
----------
0.35519981384277344, own opinion : and criminals
----------
0.35523557662963867, and women : and criminals
----------
0.35535240173339844, not opinion : and criminals
----------
0.3554638922214508, historical fact : and criminals
----------
0.3555538058280945, your choice : her self
----------
0.3555681109428406, choice something : because someone
----------
0.35571151971817017, the individual : and criminals
----------
0.3558904230594635, that decision : and criminals
----------
0.3559306263923645, judgement day : of abortions
----------
0.3560705780982971, the state : and criminals
----------
0.35612794756889343, black market : an infection
----------
0.3561416268348694, that kills : and criminals
----------
0.35625481605529785, quasi-scientific/religious opinion : are people
----------
0.35633033514022827, the fact : the people
----------
0.3563845157623291, 39 women : are people
----------
0.3563915193080902, back alley : coat hanger
----------
0.3565099239349365, botched abortions : with hate
----------
0.3565669059753418, the year : are people
----------
0.3566109538078308, the state : with hate
----------
0.35669320821762085, mandated criminalization : and criminals
----------
0.35678982734680176, the rest : the people
----------
0.35695165395736694, the debate : with hate
----------
0.3570592701435089, a nutshell : the cause
----------
validation err: 0.47520661157024796